# JPR 2020 RR notebook

This is the version of the 'master' notebook that produced results for the resubmission of the JPR 2020 article.

In [ ]:
import logging
import itertools
import multiprocessing as mp
import os
import json
import time
import copy

import numpy as np
import pandas as pd
import importlib
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn

from views.apps.plot.maps.fancy import fancy
from views.apps.ensemble import ensemble as ens, calibrate
from views.apps.ensemble import plots as ep
from views.apps.model import model as mod 
from views.apps.evaluate import evaluation, eval_tex, eval_plot, eval_data, eval_lib
from views.apps.evaluate import feature_importance as fi
from views.utils import datautils, dbutils, pyutils, config

In [ ]:
notebook_timer_start = time.time()

In [ ]:
%matplotlib inline

In [ ]:
logging.basicConfig(format=pyutils.LOGFORMAT, level=logging.INFO)
Logger = logging.getLogger(__name__)

In [ ]:
# Set notebook width to fill browser
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

# INITIAL SETUP <a name="initial_setup"></a>

In [ ]:
# Meta-data
run_id = "JPR 2020"

In [ ]:
# Feature switches

# Set True to delete dir_output and start fresh
delete_output_start_fresh = False

# Set False to use cached data files
refetch_data_from_db = False

# Leave off, ablation already worked in to ensembles I think.
do_ablation = False

# Leave on, at least for first run.
do_prediction_maps = False 

# Leave off, requires locally synced runs. 
# The function that reads the existing feature importance scores from database always runs.
redo_featimp = False 

# Plot calibration curve per constituent model step
plot_calibration_ss = True

## Choose output directory

In [ ]:
# Setup a cache directory for data so we can restart the notebook
# in 5s instead of 20 minutes in case we mess up the data
dir_scratch = config.CONFIG['dirs']['dir_scratch']
dir_cache = os.path.join(dir_scratch, "cache", "jpr_2020")
pyutils.create_dir(dir_cache)

# Setup output directory structure
#dir_output = os.path.join(dir_scratch, "output", "jpr_2020")
# Håvard: set this to overleaf dropbox dir to write output directly to overleaf.
dir_output = "/Users/havardhegre/Dropbox/Apps/ShareLaTeX/JPR_2020_R1 (Copy)/notebook_output/" 

if delete_output_start_fresh:
    pyutils.delete_dir(dir_output)

dir_plots = os.path.join(dir_output, "plots")
dir_plots_pies = os.path.join(dir_output, "plots", "pies")
dir_plots_stacks = os.path.join(dir_output, "plots", "stacks")
dir_plots_corr_pgm = os.path.join(dir_output, "plots", "corr", "pgm")
dir_plots_corr_cm = os.path.join(dir_output, "plots", "corr", "cm")
dir_plots_maps_pgm = os.path.join(dir_output, "plots", "maps", "pgm")
dir_plots_maps_cm = os.path.join(dir_output, "plots", "maps", "cm")
dir_plots_diagnostic_cm = os.path.join(dir_output, "plots", "diagnostic", "cm")
dir_plots_diagnostic_pgm = os.path.join(dir_output, "plots", "diagnostic", "pgm")
dir_plots_heatmaps_cm = os.path.join(dir_output, "plots", "heatmaps", "cm")
dir_plots_heatmaps_pgm = os.path.join(dir_output, "plots", "heatmaps", "pgm")
dir_plots_spaghetti_cm = os.path.join(dir_output, "plots", "spaghetti", "cm")
dir_plots_featimps_cm = os.path.join(dir_output, "plots", "featimp", "cm")
dir_plots_featimps_pgm = os.path.join(dir_output, "plots", "featimp", "pgm")
dir_plots_deltas_cm = os.path.join(dir_output, "plots", "deltas", "cm")
dir_plots_deltas_pgm = os.path.join(dir_output, "plots", "deltas", "pgm")
dir_plots_calibration_cm = os.path.join(dir_output, "plots", "calibration", "cm")
dir_plots_lines_cm = os.path.join(dir_output, "plots", "lines", "cm")
dir_plots_lines_pgm = os.path.join(dir_output, "plots", "lines", "pgm")

pyutils.create_dir(dir_plots)
pyutils.create_dir(dir_plots_pies)
pyutils.create_dir(dir_plots_stacks)
pyutils.create_dir(dir_plots_corr_pgm)
pyutils.create_dir(dir_plots_corr_cm)
pyutils.create_dir(dir_plots_maps_pgm)
pyutils.create_dir(dir_plots_maps_cm)
pyutils.create_dir(dir_plots_diagnostic_cm)
pyutils.create_dir(dir_plots_diagnostic_pgm)
pyutils.create_dir(dir_plots_heatmaps_cm)
pyutils.create_dir(dir_plots_heatmaps_pgm)
pyutils.create_dir(dir_plots_spaghetti_cm)
pyutils.create_dir(dir_plots_featimps_cm)
pyutils.create_dir(dir_plots_featimps_pgm)
pyutils.create_dir(dir_plots_deltas_cm)
pyutils.create_dir(dir_plots_deltas_pgm)
pyutils.create_dir(dir_plots_calibration_cm)
pyutils.create_dir(dir_plots_lines_cm)
pyutils.create_dir(dir_plots_lines_pgm)

dir_featimps_pgm = os.path.join(dir_output, "feature_importance", "pgm")
dir_featimps_cm = os.path.join(dir_output, "feature_importance", "cm")
pyutils.create_dir(dir_featimps_pgm)
pyutils.create_dir(dir_featimps_cm)

dir_tables = os.path.join(dir_output, "tables")
dir_tables_eval = os.path.join(dir_output, "tables", "eval")
dir_tables_eval_weights = os.path.join(dir_output, "tables", "eval", "weights")
dir_tables_eval_ensembles = os.path.join(dir_output, "tables", "eval", "ensembles")
dir_tables_eval_ensembles_oldvsnew = os.path.join(dir_output, "tables", "eval", "ensembles", "oldvsnew")
dir_tables_eval_ensembles_ebmavsunweighted = os.path.join(dir_output, "tables", "eval", "ensembles", "ebmavsunweighted")
dir_tables_eval_ensembles_onset = os.path.join(dir_output, "tables", "eval", "ensembles", "onset")
dir_tables_eval_ensembles_incidence = os.path.join(dir_output, "tables", "eval", "ensembles", "incidence")
dir_tables_eval_ensembles_compare_onset = os.path.join(dir_output, "tables", "eval", "ensembles", "compare_onset")
dir_tables_eval_ensembles_compare_incidence = os.path.join(dir_output, "tables", "eval", "ensembles", "compare_incidence")
dir_tables_eval_ensembles_incidencevsonset = os.path.join(dir_output, "tables", "eval", "ensembles", "incidencevsonset")
dir_tables_eval_constituent = os.path.join(dir_output, "tables", "eval", "constituent")
dir_tables_eval_constituent_onset = os.path.join(dir_output, "tables", "eval", "constituent", "onset")
dir_tables_eval_constituent_incidence = os.path.join(dir_output, "tables", "eval", "constituent", "incidence")
dir_tables_eval_constituent_compare_onset = os.path.join(dir_output, "tables", "eval", "constituent", "compare_onset")
dir_tables_eval_constituent_compare_incidence = os.path.join(dir_output, "tables", "eval", "constituent", "compare_incidence")
dir_tables_eval_constituent_incidencevsonset = os.path.join(dir_output, "tables", "eval", "constituent", "incidencevsonset")

dir_tables_featimp_cm = os.path.join(dir_output, "tables", "featimp", "cm")
dir_tables_featimp_pgm = os.path.join(dir_output, "tables", "featimp", "pgm")

pyutils.create_dir(dir_tables)
pyutils.create_dir(dir_tables_eval)
pyutils.create_dir(dir_tables_eval_weights)
pyutils.create_dir(dir_tables_eval_ensembles)
pyutils.create_dir(dir_tables_eval_ensembles_oldvsnew)
pyutils.create_dir(dir_tables_eval_ensembles_ebmavsunweighted)
pyutils.create_dir(dir_tables_eval_ensembles_onset)
pyutils.create_dir(dir_tables_eval_ensembles_incidence)
pyutils.create_dir(dir_tables_eval_ensembles_compare_onset)
pyutils.create_dir(dir_tables_eval_ensembles_compare_incidence)
pyutils.create_dir(dir_tables_eval_ensembles_incidencevsonset)

pyutils.create_dir(dir_tables_eval_constituent)
pyutils.create_dir(dir_tables_eval_constituent_onset)
pyutils.create_dir(dir_tables_eval_constituent_incidence)
pyutils.create_dir(dir_tables_eval_constituent_compare_onset)
pyutils.create_dir(dir_tables_eval_constituent_compare_incidence)
pyutils.create_dir(dir_tables_eval_constituent_incidencevsonset)
pyutils.create_dir(dir_tables_featimp_cm)
pyutils.create_dir(dir_tables_featimp_pgm)

In [ ]:
dir_tables_eval_ensembles_ablation = os.path.join(dir_output, "tables", "eval", "ensembles", "ablation")
pyutils.create_dir(dir_tables_eval_ensembles_ablation)

In [ ]:
# Set ids and actuals cols
ids_pgm = ["month_id", "pg_id"]
ids_cm = ["month_id", "country_id"]
cols_actuals = [
    "ged_tx_sb_25",
    "ged_tx_ns_25",
    "ged_tx_os_25",
    "ged_dummy_sb",
    "ged_dummy_ns",
    "ged_dummy_os",
    "onset24_tx_sb_1",
    "onset24_tx_ns_1",
    "onset24_tx_os_1",
    "onset24_tx_sb_5",
    "onset24_tx_ns_5",
    "onset24_tx_os_5",
    "onset24_tx_sb_25",
    "onset24_tx_ns_25",
    "onset24_tx_os_25"
]
periods = {
    "a": {"start": 397, "end": 433},
    "b": {"start": 433, "end": 468},
    "c": {"start": 481, "end": 518},
}

In [ ]:
%%time
# Get mapdata for plotting
mapdata = fancy.MapData()

# GET DATA <a name="get_data"></a>

In [ ]:
def get_globally_trained():
    """ Get predictions from global training notebook"""
    
    def fix_global_names(df):
        for modeltemplate in ['wdi_all', 'vdem', 'reign', 'all']:
            for step in [1, 3, 6, 12, 24, 36]:
                for outcome in ["sb", "ns", "os"]:
                    col_to_rename = 'ss.' + modeltemplate + '_' + outcome + '.' + str(step)
                    renamed = 'ss.' + outcome + '_'  + modeltemplate + '_glob' + '.' + str(step)
                    df.rename(columns={col_to_rename: renamed}, inplace=True)

        return df
    
    # Get predictions from HH Model development notebook
    df_a = dbutils.db_to_df(
            fqtable="newpipe.cm_hh_a_predict_fh",
            ids=["month_id", "country_id"],
            columns=[
                col for col in dbutils.list_columns("newpipe.cm_hh_a_predict_fh") if col.startswith("ss.")
            ],
        )

    df_b = dbutils.db_to_df(
            fqtable="newpipe.cm_hh_b_predict_fh",
            ids=["month_id", "country_id"],
            columns=[
                col for col in dbutils.list_columns("newpipe.cm_hh_b_predict_fh") if col.startswith("ss.")
            ],
        )
    df_c = dbutils.db_to_df(
            fqtable="newpipe.cm_hh_c_predict_r_2020_02_02",
            ids=["month_id", "country_id"],
            columns=[
                col for col in dbutils.list_columns("newpipe.cm_hh_c_predict_r_2020_02_02") if col.startswith("ss.")
            ],
        )
    df_a.fillna(df_a.mean(), inplace=True)
    df_b.fillna(df_b.mean(), inplace=True)
    df_c.fillna(df_c.mean(), inplace=True)
    df_a = fix_global_names(df_a)
    df_b = fix_global_names(df_b)
    df_c = fix_global_names(df_c)
    
    return df_a, df_b, df_c

In [ ]:
def get_xgboost(periods):
    """Get XGBoost predictions from flat_materialized and assign them to a/b/c """
    def reindex_groupvar(df_lacking, df_has):
        """ df_lacking is missing some groupvar index positions,
        fill them from df_has and fill the induced missingness
        with time level means.
        """
        # Make a df to put our fixed df_lacking in
        df = df_lacking.copy()
        # gids from df_has
        gids_all = sorted(list(df_has.loc[500].index.values))
        # times from df_lacking
        times_lacking = list(df.index.levels[0])
        # Make a new index
        new_ix = list(itertools.product(times_lacking, gids_all))
        # Create new index positions for the previously missing slots
        df = df.reindex(new_ix)
        # Fill with time level mean
        df = df.fillna(df.groupby(level=0).mean())
        return df

    schema = "flat_materialized"
    tables = [
        "ns_ta_pbc_1",
        "ns_ta_pbc_12",
        "ns_ta_pbc_24",
        "ns_ta_pbc_3",
        "ns_ta_pbc_36",
        "ns_ta_pbc_6",
        "ns_ta_pc_1",
        "ns_ta_pc_12",
        "ns_ta_pc_24",
        "ns_ta_pc_3",
        "ns_ta_pc_36",
        "ns_ta_pc_6",
        "ns_ta_pf_1",
        "ns_ta_pf_12",
        "ns_ta_pf_24",
        "ns_ta_pf_3",
        "ns_ta_pf_36",
        "ns_ta_pf_6",
        "ns_tab_pc_1",
        "ns_tab_pc_12",
        "ns_tab_pc_24",
        "ns_tab_pc_3",
        "ns_tab_pc_36",
        "ns_tab_pc_6",
        "ns_tab_pf_1",
        "ns_tab_pf_12",
        "ns_tab_pf_24",
        "ns_tab_pf_3",
        "ns_tab_pf_36",
        "ns_tab_pf_6",
        "ns_tabc_pf_1",
        "ns_tabc_pf_12",
        "ns_tabc_pf_24",
        "ns_tabc_pf_3",
        "ns_tabc_pf_36",
        "ns_tabc_pf_6",
        "os_ta_pbc_1",
        "os_ta_pbc_12",
        "os_ta_pbc_24",
        "os_ta_pbc_3",
        "os_ta_pbc_36",
        "os_ta_pbc_6",
        "os_ta_pc_1",
        "os_ta_pc_12",
        "os_ta_pc_24",
        "os_ta_pc_3",
        "os_ta_pc_36",
        "os_ta_pc_6",
        "os_ta_pf_1",
        "os_ta_pf_12",
        "os_ta_pf_24",
        "os_ta_pf_3",
        "os_ta_pf_36",
        "os_ta_pf_6",
        "os_tab_pc_1",
        "os_tab_pc_12",
        "os_tab_pc_24",
        "os_tab_pc_3",
        "os_tab_pc_36",
        "os_tab_pc_6",
        "os_tab_pf_1",
        "os_tab_pf_12",
        "os_tab_pf_24",
        "os_tab_pf_3",
        "os_tab_pf_36",
        "os_tab_pf_6",
        "os_tabc_pf_1",
        "os_tabc_pf_12",
        "os_tabc_pf_24",
        "os_tabc_pf_3",
        "os_tabc_pf_36",
        "os_tabc_pf_6",
        "sb_ta_pbc_1",
        "sb_ta_pbc_12",
        "sb_ta_pbc_24",
        "sb_ta_pbc_3",
        "sb_ta_pbc_36",
        "sb_ta_pbc_6",
        "sb_ta_pc_1",
        "sb_ta_pc_12",
        "sb_ta_pc_24",
        "sb_ta_pc_3",
        "sb_ta_pc_36",
        "sb_ta_pc_6",
        "sb_ta_pf_1",
        "sb_ta_pf_12",
        "sb_ta_pf_24",
        "sb_ta_pf_3",
        "sb_ta_pf_36",
        "sb_ta_pf_6",
        "sb_tab_pc_1",
        "sb_tab_pc_12",
        "sb_tab_pc_24",
        "sb_tab_pc_3",
        "sb_tab_pc_36",
        "sb_tab_pc_6",
        "sb_tab_pf_1",
        "sb_tab_pf_12",
        "sb_tab_pf_24",
        "sb_tab_pf_3",
        "sb_tab_pf_36",
        "sb_tab_pf_6",
        "sb_tabc_pf_1",
        "sb_tabc_pf_12",
        "sb_tabc_pf_24",
        "sb_tabc_pf_3",
        "sb_tabc_pf_36",
        "sb_tabc_pf_6",
    ]

    # Get dfs in dict by name
    cols = ["pg_id", "y_month", "pred_ensemble"]
    dfs = {
        table: dbutils.db_to_df(fqtable=f"{schema}.{table}", columns=cols)
        .rename(columns={"y_month": "month_id"})
        .set_index(["month_id", "pg_id"])
        .sort_index()
        for table in tables
        if all(
            (col in dbutils.list_columns(f"{schema}.{table}") for col in cols)
        )
    }

    # Translate the period names
    period_lookups = {
        "a": "ta_pbc",
        "b": "ta_pbc",
        "c": "ta_pf",
    }

    # Organise dfs by outcome/period/step in nest
    nest = dict()
    for outcome in ["sb", "ns", "os"]:
        nest[outcome] = dict()
        for period in ["a", "b", "c"]:
            nest[outcome][period] = dict()
            period_lookup = period_lookups[period]
            for step in [1, 6, 12, 24, 36]:
                nest[outcome][period][step] = dfs[f"{outcome}_{period_lookup}_{step}"]

    # Merge and rename dfs into dfs_by_period
    dfs_by_period = {"a": None, "b": None, "c": None}
    for period in ["a", "b", "c"]:
        dfs_period = []
        for step in [1, 6, 12, 24, 36]:
            for outcome in ["sb", "ns", "os"]:
                _df = nest[outcome][period][step].copy()
                _df = _df.rename(
                    columns={"pred_ensemble": f"ss.{outcome}_xgb.{step}"}
                )
                dfs_period.append(_df)
        dfs_by_period[period] = datautils.merge_dfs(dfs_period)

    # Subset by times
    df_a = dfs_by_period["a"].sort_index().loc[periods["a"]["start"]:periods["a"]["end"]]
    df_b = dfs_by_period["b"].sort_index().loc[periods["b"]["start"]:periods["b"]["end"]]
    df_c = dfs_by_period["c"].sort_index().loc[periods["c"]["start"]:periods["c"]["end"]]

    # There are grids missing for df_a and df_b that exist in df_c
    # Make sure this is the case
    assert len(df_a.loc[397]) < len(df_c.loc[481])

    df_a = reindex_groupvar(df_lacking=df_a, df_has=df_c)
    df_b = reindex_groupvar(df_lacking=df_b, df_has=df_c)

    # Fill 518 with 517 predictions
    df_t = df_c.loc[517]
    df_t["month_id"] = 518
    df_t = df_t.reset_index().set_index(["month_id", "pg_id"]).sort_index()
    df_c = df_c.append(df_t)
    df_c = df_c.sort_index()

    return df_a, df_b, df_c


In [ ]:
%%time
# Try loading cached data
try:
    # Setting refetch_data_from_db=True raises OSError, the same as if the files don't exist.
    if refetch_data_from_db:
        raise OSError 

    df_pgm_a = datautils.load_parquet(path=os.path.join(dir_cache, "df_pgm_a.parquet"))
    df_pgm_b = datautils.load_parquet(path=os.path.join(dir_cache, "df_pgm_b.parquet"))
    df_pgm_c = datautils.load_parquet(path=os.path.join(dir_cache, "df_pgm_c.parquet"))
    df_cm_a = datautils.load_parquet(path=os.path.join(dir_cache, "df_cm_a.parquet"))
    df_cm_b = datautils.load_parquet(path=os.path.join(dir_cache, "df_cm_b.parquet"))
    df_cm_c = datautils.load_parquet(path=os.path.join(dir_cache, "df_cm_c.parquet"))
    print("Loaded data from files")

# File not found gives OSError, so fetch from sources.
except OSError:
    print("failed to load from files, fetching from db")

    # Load newpipe predictions
    df_pgm_a = dbutils.db_to_df(
        fqtable="newpipe.pgm_africa_1_a_predict",
        ids=ids_pgm,
        columns=[
            col
            for col in dbutils.list_columns("newpipe.pgm_africa_1_a_predict")
            if col.startswith("memt.") or col.startswith("semt.")
        ],
    )
    df_pgm_b = dbutils.db_to_df(
        fqtable="newpipe.pgm_africa_1_b_predict",
        ids=ids_pgm,
        columns=[
            col
            for col in dbutils.list_columns("newpipe.pgm_africa_1_b_predict")
            if col.startswith("memt.") or col.startswith("semt.")
        ],
    )
    df_pgm_c = dbutils.db_to_df(
        fqtable="newpipe.pgm_africa_1_c_predict_pgm_r_2020_02_02",
        ids=ids_pgm,
        columns=[
            col
            for col in dbutils.list_columns("newpipe.pgm_africa_1_c_predict_pgm_r_2020_02_02")
            if col.startswith("memt.") or col.startswith("semt.")
        ],
    )

    df_cm_a = dbutils.db_to_df(
        fqtable="newpipe.cm_africa_1_a_predict",
        ids=ids_cm,
        columns=[
            col
            for col in dbutils.list_columns("newpipe.cm_africa_1_a_predict")
            if col.startswith("memt.") or col.startswith("semt.")
        ],
    )
    df_cm_b = dbutils.db_to_df(
        fqtable="newpipe.cm_africa_1_b_predict",
        ids=ids_cm,
        columns=[
            col
            for col in dbutils.list_columns("newpipe.cm_africa_1_b_predict")
            if col.startswith("memt.") or col.startswith("semt.")
        ],
    )
    df_cm_c = dbutils.db_to_df(
        fqtable="newpipe.cm_africa_1_c_predict_cm_r_2020_02_02",
        ids=ids_cm,
        columns=[
            col
            for col in dbutils.list_columns("newpipe.cm_africa_1_c_predict_cm_r_2020_02_02")
            if col.startswith("memt.") or col.startswith("semt.")
        ],
    )

    # join in cm actuals
    df_cm_actuals = dbutils.db_to_df(fqtable="preflight.flight_cm", ids=ids_cm, columns=cols_actuals)
    df_cm_a = df_cm_a.join(df_cm_actuals, how="inner")
    df_cm_b = df_cm_b.join(df_cm_actuals, how="inner")
    df_cm_c = df_cm_c.join(df_cm_actuals, how="inner")


    # join in pgm actuals
    df_pgm_actuals = dbutils.db_to_df(fqtable="preflight.flight_pgm", ids=ids_pgm, columns=cols_actuals)
    df_pgm_a = df_pgm_a.join(df_pgm_actuals, how="inner")
    df_pgm_b = df_pgm_b.join(df_pgm_actuals, how="inner")
    df_pgm_c = df_pgm_c.join(df_pgm_actuals, how="inner")

    # Join in dynasim results for cm
    df_cm_a = df_cm_a.join(dbutils.db_to_df(fqtable="newpipe.ds_cm_africa_1_a", ids=ids_cm,))
    df_cm_b = df_cm_b.join(dbutils.db_to_df(fqtable="newpipe.ds_cm_africa_1_b", ids=ids_cm,))
    df_cm_c = df_cm_c.join(dbutils.db_to_df(fqtable="newpipe.ds_cm_africa_1_c_r2020_02_02", ids=ids_cm,))

    # Join in dynasim results for pgm
    df_pgm_a = df_pgm_a.join(dbutils.db_to_df(fqtable="newpipe.ds_pgm_africa_1_a", ids=ids_pgm,))
    df_pgm_b = df_pgm_b.join(dbutils.db_to_df(fqtable="newpipe.ds_pgm_africa_1_b", ids=ids_pgm,))
    df_pgm_c = df_pgm_c.join(dbutils.db_to_df(fqtable="newpipe.ds_pgm_africa_1_c_r2020_02_02", ids=ids_pgm,))
    
    # Join in globally trained models
    df_cm_a_glob, df_cm_b_glob, df_cm_c_glob = get_globally_trained()
    df_cm_a = df_cm_a.join(df_cm_a_glob)
    df_cm_b = df_cm_b.join(df_cm_b_glob)
    df_cm_c = df_cm_c.join(df_cm_c_glob)
    
    # Join in xgboost predictions
    df_xgb_a, df_xgb_b, df_xgb_c = get_xgboost(periods)
    df_pgm_a = df_pgm_a.join(df_xgb_a)
    df_pgm_b = df_pgm_b.join(df_xgb_b)
    df_pgm_c = df_pgm_c.join(df_xgb_c)
    
    
    print("Read from database")
    
    # Write our cache
    datautils.write_parquet(data=df_pgm_a, path=os.path.join(dir_cache, "df_pgm_a.parquet"))
    datautils.write_parquet(data=df_pgm_b, path=os.path.join(dir_cache, "df_pgm_b.parquet"))
    datautils.write_parquet(data=df_pgm_c, path=os.path.join(dir_cache, "df_pgm_c.parquet"))
    datautils.write_parquet(data=df_cm_a, path=os.path.join(dir_cache, "df_cm_a.parquet"))
    datautils.write_parquet(data=df_cm_b, path=os.path.join(dir_cache, "df_cm_b.parquet"))
    datautils.write_parquet(data=df_cm_c, path=os.path.join(dir_cache, "df_cm_c.parquet"))
    
    print(f"Wrote cache to {dir_cache}")

In [ ]:
# Functions for fixing the naming

def shorten_names(df):
    df = df.rename(columns = lambda col: col.replace("_rf_shift_fewsteps", ""))
    return df

def rename_semt_memt_to_ss_sc(df):
    df = df.rename(columns = lambda col: col.replace("semt.", "ss."))
    df = df.rename(columns = lambda col: col.replace("memt.", "sc."))
    return df

def fix_cm_names(df):
    cm_renames = {
        ".sbonset24_" : ".sb_onset24_",
        ".nsonset24_" : ".ns_onset24_",
        ".osonset24_" : ".os_onset24_",
        ".sb100_all": ".sb_100_all",
        ".ns100_all": ".ns_100_all",
        ".os100_all": ".os_100_all",
    }
    for key in cm_renames.keys():
        for col in df.columns:
            if key in col:
                newcol = col.replace(key, cm_renames[key])
                df = df.rename(columns={col: newcol})
    return df

def copy_ds_sc_to_ss(df, cols):
    for step in [1, 6, 12, 24, 36]:
        for sc_col in cols:
            ss_col = sc_col.replace("sc.", "ss.")
            ss_col = f"{ss_col}.{step}"
            if not ss_col in df.columns:
                df[ss_col] = df[sc_col]

    return df

cols_ds_cm = [
    "sc.os_ds_25",
    "sc.sb_ds_25",
    "sc.ns_ds_25",
    "sc.sb_ds_dummy",
    "sc.ns_ds_dummy",
    "sc.os_ds_dummy",
    "sc.any_ds_500",
]
cols_ds_pgm = [
    "sc.os_ds_25",
    "sc.sb_ds_25",
    "sc.ns_ds_25",
    "sc.sb_ds_dummy",
    "sc.ns_ds_dummy",
    "sc.os_ds_dummy",
]

In [ ]:
%%time
# Reload data we cached
df_pgm_a = datautils.load_parquet(path=os.path.join(dir_cache, "df_pgm_a.parquet"))
df_pgm_b = datautils.load_parquet(path=os.path.join(dir_cache, "df_pgm_b.parquet"))
df_pgm_c = datautils.load_parquet(path=os.path.join(dir_cache, "df_pgm_c.parquet"))
df_cm_a = datautils.load_parquet(path=os.path.join(dir_cache, "df_cm_a.parquet"))
df_cm_b = datautils.load_parquet(path=os.path.join(dir_cache, "df_cm_b.parquet"))
df_cm_c = datautils.load_parquet(path=os.path.join(dir_cache, "df_cm_c.parquet"))

# And fix the names
df_cm_a = shorten_names(df_cm_a)
df_cm_b = shorten_names(df_cm_b)
df_cm_c = shorten_names(df_cm_c)
df_pgm_a = shorten_names(df_pgm_a)
df_pgm_b = shorten_names(df_pgm_b)
df_pgm_c = shorten_names(df_pgm_c)
df_cm_a = fix_cm_names(df_cm_a)
df_cm_b = fix_cm_names(df_cm_b)
df_cm_c = fix_cm_names(df_cm_c)
df_cm_a = rename_semt_memt_to_ss_sc(df_cm_a)
df_cm_b = rename_semt_memt_to_ss_sc(df_cm_b)
df_cm_c = rename_semt_memt_to_ss_sc(df_cm_c)
df_pgm_a = rename_semt_memt_to_ss_sc(df_pgm_a)
df_pgm_b = rename_semt_memt_to_ss_sc(df_pgm_b)
df_pgm_c = rename_semt_memt_to_ss_sc(df_pgm_c)
df_pgm_a = copy_ds_sc_to_ss(df_pgm_a, cols_ds_pgm)
df_pgm_b = copy_ds_sc_to_ss(df_pgm_b, cols_ds_pgm)
df_pgm_c = copy_ds_sc_to_ss(df_pgm_c, cols_ds_pgm)
df_cm_a = copy_ds_sc_to_ss(df_cm_a, cols_ds_cm)
df_cm_b = copy_ds_sc_to_ss(df_cm_b, cols_ds_cm)
df_cm_c = copy_ds_sc_to_ss(df_cm_c, cols_ds_cm)

In [ ]:
%%time
# Compute crosslevel prediction
def fetch_df_links():
    """Get a df linking pg_ids to country_ids."""

    query = """
    SELECT pgm.priogrid_gid AS pg_id,
       cm.country_id
    FROM staging.priogrid_month AS pgm
         INNER JOIN staging.country_month AS cm ON pgm.country_month_id = cm.id
    --- Month 500 arbitrary choice
    WHERE pgm.month_id = 500;
    """
    return dbutils.query_to_df(query)

def compute_colaresi(df, col_pgm, col_cm):
    """ Colaresian cross level probability """

    # Sum of high resolution probabilities for each low level area
    sum_h_by_l = df.groupby(["month_id", "country_id"])[col_pgm].transform(sum)
    
    # Low resolution prob multiplied by share of high res prob in particular area
    joint_prob = df[col_cm] * (df[col_pgm] / sum_h_by_l)

    return joint_prob

def crosslevel(df_pgm, df_cm, df_links, col_pgm, col_cm):
    # Join in country_id
    df = df_pgm[[col_pgm]].join(df_links.set_index(["pg_id"])[["country_id"]])
    df = df.reset_index().set_index(["month_id", "country_id"])
    df = df.join(df_cm[[col_cm]]).reset_index().set_index(["month_id", "pg_id"])
    s = compute_colaresi(df, col_pgm, col_cm)
    share_missing=s.isnull().sum()/len(s)
    if share_missing > 0.01:
        raise RuntimeError(f"Too much missing in prediction, something's wrong")
    s = s.fillna(s.mean())
    return s


df_links = fetch_df_links()
for step in [1, 6, 12, 24, 36]:
    for outcome in ["sb", "ns", "os"]:
        col_cl = f"ss.{outcome}_crosslevel.{step}"
        col_pgm = f"ss.{outcome}_xgb.{step}" # Use the allthemes model for pgm
        col_cm = f"ss.{outcome}_all_glob.{step}" # Use the all_glob model for CM
        df_pgm_a[col_cl] = crosslevel(df_pgm_a, df_cm_a, df_links, col_pgm, col_cm)
        df_pgm_b[col_cl] = crosslevel(df_pgm_b, df_cm_b, df_links, col_pgm, col_cm)
        df_pgm_c[col_cl] = crosslevel(df_pgm_c, df_cm_c, df_links, col_pgm, col_cm)
        

# DEFINE MODELS AND ENSEMBLES

In [ ]:
# Define the base models that exist for each outcome
basemodels_pgm = [
    "allthemes",
    "hist_legacy",
    "onset24_100_all",
    "onset24_1_all",
    "pgd_natural",
    "pgd_social",
    "spei_full",
    "sptime",
    "ds_25",
    "ds_dummy",
    "xgb",
    "crosslevel",
]

basemodels_cm = [
    "cflong", 
    "acled_violence", 
    "neibhist", 
    "cdummies", 
    "acled_protest", 
    "reign_coups", 
    "icgcw", 
    "reign_drought", 
    "reign_glob",
    "vdem_glob",
    "demog", 
    "wdi_all_glob",
    "ds_25",
    "ds_dummy",
    "all_glob",
    "onset24_25_all", 
]

In [ ]:
# Define per-outcome ensemble basemodel lists
basemodels_cm_sb = [
    "cflong", 
    "acled_violence", 
    "neibhist", 
    "cdummies", 
    "acled_protest", 
    "reign_coups", 
    "icgcw", 
    "reign_drought", 
    "reign_glob",
    "vdem_glob",
    "demog", 
    "wdi_all_glob",
    "ds_25",
    "ds_dummy",
    "all_glob",
    "onset24_25_all", 
]

basemodels_cm_ns = [
    "cflong", 
    "acled_violence", 
    "neibhist", 
    "cdummies", 
    "acled_protest", 
    "reign_coups", 
    "icgcw", 
    "reign_drought", 
    "reign_glob",
    "vdem_glob",
    "demog", 
    "wdi_all_glob",
    "ds_25",
    "ds_dummy",
    "all_glob",
    "onset24_25_all", 
]

basemodels_cm_os = [
    "cflong", 
    "acled_violence", 
    "neibhist", 
    "cdummies", 
    "acled_protest", 
    "reign_coups", 
    "icgcw", 
    "reign_drought", 
    "reign_glob",
    "vdem_glob",
    "demog", 
    "wdi_all_glob",
    "ds_25",
    "ds_dummy",
    "all_glob",
    "onset24_25_all", 
]

# Dropped spei_full due to low performance
basemodels_pgm_sb = [
    "allthemes",
    "hist_legacy",
    "onset24_100_all",
    "onset24_1_all",
    "pgd_natural",
    "pgd_social",
    #"spei_full",
    "sptime",
    "ds_25",
    "ds_dummy",
    "xgb",
    "crosslevel",
]
basemodels_pgm_ns = [
    "allthemes",
    #"hist_legacy",
    "onset24_100_all",
    "onset24_1_all",
    "pgd_natural",
    "pgd_social",
    #"spei_full",
    "sptime",
    "ds_25",
    "ds_dummy",
    "xgb",
    "crosslevel",
]

basemodels_pgm_os = [
    "allthemes",
    "hist_legacy",
    "onset24_100_all",
    "onset24_1_all",
    "pgd_natural",
    "pgd_social",
    #"spei_full",
    "sptime",
    "ds_25",
    "ds_dummy",
    "xgb",
    "crosslevel",
]

In [ ]:
# Define constituent models dictionaries for CM + PGM
# Use the full list of models, not the per-outcome subsets
steps = [1, 6, 12, 24, 36]
outcomes = ["sb", "ns", "os"]

models_pgm = list()
for outcome in outcomes:
    for basename in basemodels_pgm:
        name = f"{outcome}_{basename}"
        col_actual = f"ged_dummy_{outcome}"
        models_pgm.append(mod.make_model_dict(name, col_actual, steps, outcome))

models_cm = list()
for outcome in outcomes:
    for basename in basemodels_cm:
        name = f"{outcome}_{basename}"
        col_actual = f"ged_tx_{outcome}_25"
        models_cm.append(mod.make_model_dict(name, col_actual, steps, outcome))


In [ ]:
# Define CM ensembles

# 25 BRD
cm_all_sb = ens.make_ensemble_dict(
    name = "sb_ensemble_all",
    col_actual = "ged_tx_sb_25",
    outcome = "sb",
    models = [f"sb_{col}" for col in basemodels_cm_sb],
    steps = [1,6, 12, 24, 36],
)

cm_all_ns = ens.make_ensemble_dict(
    name = "ns_ensemble_all",
    col_actual = "ged_tx_ns_25",
    outcome = "ns",
    models = [f"ns_{col}" for col in basemodels_cm_ns],
    steps = [1,6, 12, 24, 36],
)

cm_all_os = ens.make_ensemble_dict(
    name = "os_ensemble_all",
    col_actual = "ged_tx_os_25",
    outcome = "os",
    models = [f"os_{col}" for col in basemodels_cm_os],
    steps = [1,6, 12, 24, 36],
)

# Single BRD
cm_all_1brd_sb = ens.make_ensemble_dict(
    name = "sb_ensemble_1brd",
    col_actual = "ged_dummy_sb",
    outcome = "sb",
    models = [f"sb_{col}" for col in basemodels_cm_sb],
    steps = [1,6, 12, 24, 36],
)

cm_all_1brd_ns = ens.make_ensemble_dict(
    name = "ns_ensemble_1brd",
    col_actual = "ged_dummy_ns",
    outcome = "ns",
    models = [f"ns_{col}" for col in basemodels_cm_ns],
    steps = [1,6, 12, 24, 36],
)

cm_all_1brd_os = ens.make_ensemble_dict(
    name = "os_ensemble_1brd",
    col_actual = "ged_dummy_os",
    outcome = "os",
    models = [f"os_{col}" for col in basemodels_cm_os],
    steps = [1,6, 12, 24, 36],
)

ensembles_cm = [cm_all_sb, cm_all_ns, cm_all_os, cm_all_1brd_sb, cm_all_1brd_ns, cm_all_1brd_os]

# Define PGM ensembles
pgm_all_sb = ens.make_ensemble_dict(
    name = "sb_ensemble_all",
    col_actual = "ged_dummy_sb",
    outcome = "sb",
    models = [f"sb_{col}" for col in basemodels_pgm_sb],
    steps = [1,6, 12, 24, 36]
)

pgm_all_ns = ens.make_ensemble_dict(
    name = "ns_ensemble_all",
    col_actual = "ged_dummy_ns",
    outcome = "ns",
    models = [f"ns_{col}" for col in basemodels_pgm_ns],
    steps = [1,6, 12, 24, 36],
)

pgm_all_os = ens.make_ensemble_dict(
    name = "os_ensemble_all",
    col_actual = "ged_dummy_os",
    outcome = "os",
    models = [f"os_{col}" for col in basemodels_pgm_os],
    steps = [1,6, 12, 24, 36],
)

ensembles_pgm = [pgm_all_sb, pgm_all_ns, pgm_all_os]

# CALIBRATE MODELS AND COMPUTE ENSEMBLES

In [ ]:
%%time
# CM EBMA
for ensemble in ensembles_cm:

    # Get step specific weights and EBMA predictions for A->B and B->C
    ensemble['weights_ebma_ab'], df_ebma_ss_b = ens.ss_ebma(ensemble, df_cm_a, df_cm_b)
    ensemble['weights_ebma_bc'], df_ebma_ss_c = ens.ss_ebma(ensemble, df_cm_b, df_cm_c)
    
    # Join in step specific EBMA predictions for B and C
    # Doing it col by col to remain idempotent-> cell can be re-run
    for col in df_ebma_ss_b:
        df_cm_b[col] = df_ebma_ss_b[col]
    for col in df_ebma_ss_c:
        df_cm_c[col] = df_ebma_ss_c[col]
    
    # Select+Interpolate step specific EBMA predictions 
    # into step combined EBMA predictions for B and C (forecast)
    s_ebma_sc_b = ens.sc_ebma(ensemble, df_cm_b)
    df_cm_b[s_ebma_sc_b.name] = s_ebma_sc_b
    s_ebma_sc_c = ens.sc_ebma(ensemble, df_cm_c)
    df_cm_c[s_ebma_sc_c.name] = s_ebma_sc_c
    

In [ ]:
%%time
# CM Average
for ensemble in ensembles_cm:

    # Compute step specific unweighted average of calibrated constituent models
    df_avg_b = ens.ss_calibrated_unweighted_average(ensemble, df_cm_a, df_cm_b)
    df_avg_c = ens.ss_calibrated_unweighted_average(ensemble, df_cm_b, df_cm_c)
    
    # Join in averages
    for col in df_avg_b:
        df_cm_b[col] = df_avg_b[col]
    for col in df_avg_c:
        df_cm_c[col] = df_avg_c[col]

    
    # Step combined
    s_avg_b = ens.sc_calibrated_unweighted_average(
        ensemble = ensemble,
        df_calib = df_cm_a,
        df_test = df_cm_b,
    )
    df_cm_b[s_avg_b.name] = s_avg_b
    
    s_avg_c = ens.sc_calibrated_unweighted_average(
        ensemble = ensemble,
        df_calib = df_cm_b,
        df_test = df_cm_c,
    )
    df_cm_c[s_avg_c.name] = s_avg_c   
    

In [ ]:
%%time
# PGM Average
for ensemble in ensembles_pgm:
    # Compute step specific unweighted average of calibrated constituent models
    df_avg_b = ens.ss_calibrated_unweighted_average(ensemble, df_pgm_a, df_pgm_b)
    df_avg_c = ens.ss_calibrated_unweighted_average(ensemble, df_pgm_b, df_pgm_c)
    
    # Insert into our dataframes
    for col in df_avg_b:
        df_pgm_b[col] = df_avg_b[col]
        df_pgm_c[col] = df_avg_c[col]

    
    # Compute step-combined averages by selecting+interpolating step specific
    s_avg_b = ens.sc_calibrated_unweighted_average(
        ensemble = ensemble,
        df_calib = df_pgm_a,
        df_test = df_pgm_b,
    )
    df_pgm_b[s_avg_b.name] = s_avg_b
    s_avg_c = ens.sc_calibrated_unweighted_average(
        ensemble = ensemble,
        df_calib = df_pgm_b,
        df_test = df_pgm_c,
    )
    df_pgm_c[s_avg_c.name] = s_avg_c
    
        
        

In [ ]:
%%time
# Calibrate CM constituent models
df_cm_b_calibrated = pd.DataFrame(index=df_cm_b.index)
df_cm_c_calibrated = pd.DataFrame(index=df_cm_c.index)

for model in models_cm:
    for step in model['steps']:
        col_ss = model['cols_ss'][step]
        col_actual = model['col_actual']
        df_model_calibrated_b = calibrate.calibrate_models(
            df_calib = df_cm_a[[col_ss]],
            df_test = df_cm_b[[col_ss]],
            s_calib_actual=df_cm_a[col_actual]
        )
        
        df_model_calibrated_c = calibrate.calibrate_models(
            df_calib = df_cm_b[[col_ss]],
            df_test = df_cm_c[[col_ss]],
            s_calib_actual=df_cm_b[col_actual]
        )
        df_cm_b_calibrated[col_ss] = df_model_calibrated_b[col_ss]    
        df_cm_c_calibrated[col_ss] = df_model_calibrated_c[col_ss]
    
    df_cm_b_calibrated[model["col_sc"]] = mod.compute_sc_constituent(df_cm_b_calibrated, model)
    df_cm_c_calibrated[model["col_sc"]] = mod.compute_sc_constituent(df_cm_c_calibrated, model)

In [ ]:
%%time
# Calibrate PGM
df_pgm_b_calibrated = pd.DataFrame(index=df_pgm_b.index)
df_pgm_c_calibrated = pd.DataFrame(index=df_pgm_c.index)

for model in models_pgm:
    for step in model['steps']:
        col_ss = model['cols_ss'][step]
        col_actual = model['col_actual']
        df_model_calibrated_b = calibrate.calibrate_models(
            df_calib = df_pgm_a[[col_ss]],
            df_test = df_pgm_b[[col_ss]],
            s_calib_actual=df_pgm_a[col_actual]
        )
        df_model_calibrated_c = calibrate.calibrate_models(
            df_calib = df_pgm_b[[col_ss]],
            df_test = df_pgm_c[[col_ss]],
            s_calib_actual=df_pgm_b[col_actual]
        )
        df_pgm_b_calibrated[col_ss] = df_model_calibrated_b[col_ss]    
        df_pgm_c_calibrated[col_ss] = df_model_calibrated_c[col_ss]   
    
    df_pgm_b_calibrated[model["col_sc"]] = mod.compute_sc_constituent(df_pgm_b_calibrated, model)
    df_pgm_c_calibrated[model["col_sc"]] = mod.compute_sc_constituent(df_pgm_c_calibrated, model)

In [ ]:
%%time
# Ablation CM, use EBMA
# Write ablation results to ensemble dicts
if do_ablation:
    with mp.Pool(processes=4) as pool:
        results = []
        for ensemble in ensembles_cm:
            results.append(pool.apply_async(ens.ablation_study, (ensemble, df_cm_a, df_cm_b, "ebma")))
        for ensemble, result in zip(ensembles_cm, results):
            ensemble["ablation_results"] = result.get()
            

In [ ]:
%%time
# Ablation PGM, use simple average
if do_ablation:
    for ensemble in ensembles_pgm:
        ensemble["ablation_results"] = ens.ablation_study(ensemble, df_pgm_a, df_pgm_b, "average")

In [ ]:
#ensembles_pgm[0]["ablation_results"]

In [ ]:
#for ensemble in ensembles_cm:
#    ensemble['ablation_results_restored'] = pd.DataFrame.to_dict( ensemble['ablation_results'],orient='index')
#ensemble['ablation_results_restored']

In [ ]:
# Convert the results dictionary into two dataframes per ensemble
# Change to ensemble["ablation_results"] when rerunning
for ensemble in ensembles_cm:
    print(ensemble['name'])
    # Extract aupr from the dictionary inside the df:
    ensemble['ablation_aupr'] = pd.DataFrame.from_dict(ensemble["ablation_results"],orient='index')
    for model,value in ensemble['ablation_aupr'].iterrows():
        for step,j in value.iteritems():
            ensemble['ablation_aupr'].loc[model][step] = j['aupr'].astype(float)
    # Extract auroc from the dictionary inside the df:
    ensemble['ablation_auroc'] = pd.DataFrame.from_dict(ensemble["ablation_results"],orient='index')
    for model,value in ensemble['ablation_auroc'].iterrows():
        for step,j in value.iteritems():
            ensemble['ablation_auroc'].loc[model][step] = j['auroc'].astype(float)
    # Output results
    # AUPR
    filename = dir_tables_eval_ensembles_ablation + '/' + ensemble['name'] + '_aupr.tex'
    file=open(filename,'w')
    table = ensemble['ablation_aupr'].to_latex(float_format="{:0.4f}".format, column_format='lrrrrr')
    file.write(table)
    # AUROC
    filename = dir_tables_eval_ensembles_ablation + '/' + ensemble['name'] + '_auroc.tex'
    file=open(filename,'w')
    table = ensemble['ablation_auroc'].to_latex(float_format="{:0.4f}".format, column_format='lrrrrr')
    file.write(table)

In [ ]:
for ensemble in ensembles_pgm:
    print(ensemble['name'])
    # Extract aupr from the dictionary inside the df:
    ensemble['ablation_aupr'] = pd.DataFrame.from_dict(ensemble["ablation_results"],orient='index')
    for model,value in ensemble['ablation_aupr'].iterrows():
        for step,j in value.iteritems():
            ensemble['ablation_aupr'].loc[model][step] = j['aupr'].astype(float)
    # Extract auroc from the dictionary inside the df:
    ensemble['ablation_auroc'] = pd.DataFrame.from_dict(ensemble["ablation_results"],orient='index')
    for model,value in ensemble['ablation_auroc'].iterrows():
        for step,j in value.iteritems():
            ensemble['ablation_auroc'].loc[model][step] = j['auroc'].astype(float)

# PREDICTION PLOTS

## Correlation plots

In [ ]:
%%time
# Correlation plots for all models by outcome
def make_cols_by_outcome_dict(models):
    cols_by_outcome = dict()
    for model in models:
        outcome = model["outcome"]
        if not outcome in cols_by_outcome.keys():
            cols_by_outcome[outcome] = dict()
        for step in model["cols_ss"]:
            if not step in cols_by_outcome[outcome]:
                cols_by_outcome[outcome][step] = []
            cols_by_outcome[outcome][step].append(model["cols_ss"][step])
    return cols_by_outcome
    
def draw_corrplot(df, cols, title, path):
    corr = df[cols].corr()
    f, ax = plt.subplots(figsize=(11, 9))
    sns.set(style="white")
    cmap = sns.diverging_palette(220, 10, as_cmap=True)
    cmap = "Spectral_r"
    sns.heatmap(
        corr, 
        mask=False, 
        cmap=cmap, 
        vmin=-.1, 
        vmax=1, 
        center=.5,
        cbar_kws = {"shrink": .8}
    )
    ax.set_title(title)
    plt.tight_layout()
    plt.savefig(path)
    plt.close()
    
def draw_corrplots(df, models, dir_destination):
    pyutils.create_dir(dir_destination)
    cols_by_outcome = make_cols_by_outcome_dict(models)
    for outcome, cols_by_step in cols_by_outcome.items():
        for step, cols in cols_by_step.items():
            draw_corrplot(
                df, 
                cols, 
                title=f"Correlation Plot, {outcome.upper()}, step {step}", 
                path=os.path.join(dir_destination, f"{outcome}_{step}.png")
            )            

draw_corrplots(df_pgm_b, models_pgm, dir_plots_corr_pgm)
draw_corrplots(df_cm_b, models_cm, dir_plots_corr_cm)

## Prediction maps

In [ ]:
%%time
def plot_prediction_maps(
    models, key_prediction, df, times, dir_destination, mapdata,
):
    for model in models:
        for t in times:
            step = t - min(times) + 1 #the first t in times gets step=1
            title = f"{model['name']}"
            fancy.plot_map(
                mapdata=mapdata,
                s_patch=df[model[key_prediction]],
                t=t,
                run_id="JPR 2020",
                title=f"{model['name']}, step {step}",
                logodds=True,
                path=os.path.join(dir_destination, f"{model['name']}_{t}.png"),
                bbox="mainland_africa",
                s_marker=df[model["col_actual"]],
                textsize=18
            )


times_map_start_b = periods["b"]["start"]
times_map_start_c = periods["c"]["start"]
# Plot for steps 1, 3, 6, 12, 24, 36
times_map_b = [times_map_start_b, times_map_start_b+2, times_map_start_b+5, times_map_start_b+11, times_map_start_b+23, times_map_start_b+35]
times_map_c = [times_map_start_c, times_map_start_c+2, times_map_start_c+5, times_map_start_c+11, times_map_start_c+23, times_map_start_c+35]
# Short version: Plot for steps 1, 3, 36 
times_map_b_short = [times_map_start_b, times_map_start_b+2, times_map_start_b+35]
times_map_c_short = [times_map_start_c, times_map_start_c+2, times_map_start_c+35]


if do_prediction_maps:
    # B CM EBMA
    plot_prediction_maps(
        models=ensembles_cm,
        key_prediction="col_ebma_sc",
        df=df_cm_b,
        times=times_map_b,
        dir_destination=dir_plots_maps_cm,
        mapdata=mapdata,
    )
    # B CM calibrated constituent
    plot_prediction_maps(
        models=models_cm,
        key_prediction="col_sc",
        df=df_cm_b_calibrated.join(df_cm_b[cols_actuals]),
        times=times_map_b_short,
        dir_destination=dir_plots_maps_cm,
        mapdata=mapdata,
    )
    # B PGM AVG
    plot_prediction_maps(
        models=ensembles_pgm,
        key_prediction="col_avg_sc",
        df=df_pgm_b,
        times=times_map_b,
        dir_destination=dir_plots_maps_pgm,
        mapdata=mapdata,
    )
    # B PGM calibrated constituent
    plot_prediction_maps(
        models=models_pgm,
        key_prediction="col_sc",
        df=df_pgm_b_calibrated.join(df_pgm_b[cols_actuals]),
        times=times_map_b_short,
        dir_destination=dir_plots_maps_pgm,
        mapdata=mapdata,
    )

    # C CM EBMA
    plot_prediction_maps(
        models=ensembles_cm,
        key_prediction="col_ebma_sc",
        df=df_cm_c,
        times=times_map_c,
        dir_destination=dir_plots_maps_cm,
        mapdata=mapdata,
    )
    # C CM calibrated constituent
    plot_prediction_maps(
        models=models_cm,
        key_prediction="col_sc",
        df=df_cm_c_calibrated.join(df_cm_c[cols_actuals]),
        times=times_map_c_short,
        dir_destination=dir_plots_maps_cm,
        mapdata=mapdata,
    )
    # C PGM AVG
    plot_prediction_maps(
        models=ensembles_pgm,
        key_prediction="col_avg_sc",
        df=df_pgm_c,
        times=times_map_c,
        dir_destination=dir_plots_maps_pgm,
        mapdata=mapdata,
    )
    # C PGM calibrated constituent
    plot_prediction_maps(
        models=models_pgm,
        key_prediction="col_sc",
        df=df_pgm_c_calibrated.join(df_pgm_c[cols_actuals]),
        times=times_map_c_short,
        dir_destination=dir_plots_maps_pgm,
        mapdata=mapdata,
    )

## Ensemble contributions

### Ensemble weight tables

In [ ]:
# Write weights to table.
for ensemble in ensembles_cm:
    name = ensemble["name"]
    weights_df = pd.DataFrame({"Model": ensemble["models"]})
    for step, column in ensemble["weights_ebma_bc"].items():
        weights_df[f"Step {step}"] = [round(value, 4) for var, value in column.items()]

    path = os.path.join(dir_tables_eval_weights, f"ebma_weights_{name}.tex")
    with open(path, 'w') as tab:
         tab.write(weights_df.to_latex(index=False))

### Pies

In [ ]:
country_ids_to_plot =  [
    47, # Burkina Faso
    50, # Mali
    69, # Cameroon
    79, # Nigeria
    120, # Somalia
    162, # Mozambique
    158, # Zimbabwe
    213, # Libya
    222, # Egypt
    167, # DRC
]
steps_pie = [1, 12, 36]
# Make 
for ensemble in ensembles_cm:
    for country_id in country_ids_to_plot:
        # Plot the pies
        for step in steps_pie:
            df_pie = ep.join_month_ids(ep.join_country_names(ep.sc_proportion(ensemble=ensemble, df=df_cm_c_calibrated)))
            path = os.path.join(dir_plots_pies, f"{ensemble['name']}_{country_id}_s{step}")
            ep.plot_pie(df1=df_pie, df2=df_cm_c[ensemble["col_ebma_ss"][step]], models = ensemble["models"], step=step, country_id=country_id, path=path, run_id=run_id)
        
        # Plot the stacked
        df_stack = ep.join_month_ids(ep.join_country_names(ep.sc_weighted(ensemble=ensemble, df=df_cm_c_calibrated))) #Compute weighted probabilities
        path = os.path.join(dir_plots_stacks, f"{ensemble['name']}_{country_id}")
        ep.plot_stacked(df=df_stack, models=ensemble['models'], step_start=1, step_end=36, country_id=country_id, path=path, run_id=run_id)
        

In [ ]:
for ensemble in ensembles_cm:
    for country_id in country_ids_to_plot:
        
        # Plot the stacked grid
        df_stack = ep.join_month_ids(ep.join_country_names(ep.sc_weighted(ensemble=ensemble, df=df_cm_c_calibrated))) #Compute weighted probabilities
        path = os.path.join(dir_plots_stacks, f"{ensemble['name']}_grid")
        ep.plot_stacked_grid(df=df_stack, 
                        models=ensemble['models'], 
                        step_start=1, 
                        step_end=36, 
                        country_id=79, # Nigeria
                        country_id1=47, # Burkina Faso
                        country_id2=222, # Egypt
                        country_id3=162, # DRC
                        path=path, run_id=run_id)

### Spaghetti

In [ ]:
# Make spaghetti plots for all countries / highlight selected countries
countries_to_plot = [
    "Burkina Faso",
    "Cameroon",
    "Congo, DRC",
    "Egypt",
    "Libya",
    "Mali",
    "Mozambique",
    "Nigeria",
    "Somalia",
    "Sudan"
]

for ensemble in ensembles_cm:
    ep.plot_spaghetti_countries(
        df=ep.join_month_ids(ep.join_country_names(df=df_cm_b)), 
        model=ensemble["col_ebma_sc"],
        countries = countries_to_plot,
        step_start=1, 
        step_end=36,
        path=os.path.join(dir_plots_spaghetti_cm, f"{ensemble['name']}_b.png") ,
        run_id=run_id
        )
    ep.plot_spaghetti_countries(
        df=ep.join_month_ids(ep.join_country_names(df=df_cm_c)), 
        model=ensemble["col_ebma_sc"],
        countries = countries_to_plot,
        step_start=1, 
        step_end=36,
        path=os.path.join(dir_plots_spaghetti_cm, f"{ensemble['name']}_c.png"),
        run_id=run_id
        )

### Cumulative plots

In [ ]:
# Make cumulative plots for selected countires
for ensemble in ensembles_cm:
    ep.plot_cum(
        df=ep.join_month_ids(ep.join_country_names(df=df_cm_c)), 
        model=ensemble["col_ebma_sc"],
        countries = [47,  # Burkina Faso
                     50,  # Mali
                     69,], #Cameroon
        step_start=1, 
        step_end=36,
        path=os.path.join(dir_plots_lines_cm, f"cum_{ensemble['name']}_c.png"),
        run_id=run_id
        )
    

### Line plots

In [ ]:
# Make lineplots for selected countires
for ensemble in ensembles_cm:
    ep.plot_prob(
        df=ep.join_month_ids(ep.join_country_names(df=df_cm_c)), 
        model=ensemble["col_ebma_sc"],
        countries = [47,  # Burkina Faso
                     50,  # Mali
                     69,], #Cameroon
        step_start=1, 
        step_end=36,
        path=os.path.join(dir_plots_lines_cm, f"prob_{ensemble['name']}_c.png") ,
        run_id=run_id
        )

## Calibration plots

In [ ]:
%%time
run_id = run_id

# Calibration, old vs. new (skipping 25brd).
for model in ensembles_cm:
    if "ged_dummy" in model["col_actual"]: 
        for step in model["steps"]:
            col_sc = model["col_ebma_sc"]
            col_actual = model["col_actual"]
            outcome = model["outcome"]
            actuals = df_cm_b[col_actual].values
            probs = df_cm_b[col_sc].values
            col_sc = col_sc.replace(".", "_")
            eval_plot.plot_calibration_curve(
                actuals,
                probs,
                title=f"calibration, {col_sc}",
                n_bins=20,
                to_path=os.path.join(
                    dir_plots_calibration_cm, 
                    f"calib_{col_sc}.png"
                ),
            )

            current = eval_data.fetch_ensemble_predictions(
                run_id=run_id, 
                level="cm", 
                outcome=outcome, 
                eval_test=True
            )
            col_current = current.columns[0]
            probs = current[col_current].values
            col_current = col_current.replace(".", "_")
            eval_plot.plot_calibration_curve(
                actuals,
                probs,
                title=f"calibration, {col_current}",
                n_bins=20,
                to_path=os.path.join(
                    dir_plots_calibration_cm, 
                    f"calib_{col_current}.png"
                ),
            )

In [ ]:
%%time
# Calibration curve per constituent model-step.
if plot_calibration_ss:
    for model in models_cm:
        for step in model["steps"]:
            col_ss = model["cols_ss"][step]
            col_actual = model["col_actual"]
            actuals = df_cm_b[col_actual].values
            probs = df_cm_b_calibrated[col_ss].values
            col_ss = col_ss.replace(".", "_")
            eval_plot.plot_calibration_curve(
                actuals,
                probs,
                title=f"calibration curve, {col_ss}",
                n_bins=20,
                to_path=os.path.join(
                    dir_plots_calibration_cm, 
                    f"calib_{col_ss}.png"
                ),
            )

# EVALUATION

## E.1 Scores to dicts
All evaluation data is prepared in this section. All following sections draw from the dictionaries.

In [ ]:
%%time
# Proposed new metrics organisation:
# Scores collected for both ss and sc. Initially I had written some repeated code below to get the scores for sc.
# Threshold now only collected for the ensembles (ss step 1 ONLY, sc) to reduce runtime (down to approx. 12 minutes).
# Note that "threshold" remains an argument under get_metrics().

# Produces:
# - cm ensemble scores for ss
# - cm ensemble scores for sc
# - pgm ensemble scores for ss
# - pgm ensemble scores for sc
# - cm constituent scores for ss
# - pgm constituent scores for ss

def get_optimal_threshold(model, key_pred, key_actual, df_pred, df_actual, n_steps, sc=False):
    """Returns optimal threshold for a single selected model.""" 
    costs = np.array([[0.0, 1.0], [10.0, 0.0]])     
    # NOTE HARDCODED STEP 1 HERE IF SS.
    probs = df_pred[model[key_pred]] if sc else df_pred[model[key_pred][1]]
    actuals = df_actual[key_actual]
    print(f"Computing optimal threshold for {key_pred} at {key_actual} with n_steps {n_steps}...")
    optimal_threshold = eval_lib.compute_optimal_threshold(
        actuals=actuals, 
        probs=probs, 
        cost_matrix=costs, 
        parallel=False,
        n_steps=n_steps,
        )[1]
    return optimal_threshold


def get_metrics(model, key_pred, key_actual, df_pred, df_actual, n_steps=11, 
                sc=False, threshold=None, aupr_decimals=3, brier_decimals=4):
    print(f"{model['name']} " + f"{key_actual}", end = " ")
    costs = np.array([[0.0, 1.0], [10.0, 0.0]])
    
    scores = dict()
    steps = ["step_combined"] if sc else model["steps"]
          
    for step in steps:
        print(step, end=" ")
        probs = df_pred[model[key_pred]] if sc else df_pred[model[key_pred][step]]
        actuals = df_actual[key_actual]
        
        if threshold is None:
            optimal_threshold = eval_lib.compute_optimal_threshold(
                actuals, probs, costs, n_steps,
            )[1]
        else:
            optimal_threshold = threshold
        
        preds = np.where(probs >= optimal_threshold, 1, 0)

        confusion_matrix = eval_lib.confusion_matrix_by_threshold(actuals, probs, optimal_threshold)
        tn, fp, fn, tp = confusion_matrix.ravel()
        
        scores_step = {
            "AUPR": eval_lib.average_precision(actuals, probs),
            "AUROC": eval_lib.area_under_roc(actuals, probs),
            "Brier": eval_lib.brier(actuals, probs),
            "Accuracy": eval_lib.accuracy(actuals, preds),
            "F1-score": eval_lib.f1_score(actuals, preds),
            "Cost-based threshold": optimal_threshold,
            "True positive": tp,
            "False positive": fp,
            "True negative": tn,
            "False negative": fn,
        }
        
        # Insert by step key if ss.
        if sc is True:
            scores = scores_step
        else:
            scores[step] = scores_step
    
    return scores

In [ ]:
%%time
print("___ Getting the cm ensemble scores for ss... ___")
for ensemble in ensembles_cm:
    outcome = ensemble["outcome"]
    
    # Get the optimal thresholds (25brd here). Ensemble thresholds used for constituent model evaluation below.
    if "_1brd" not in ensemble["name"]:
        cm_ss_threshold = get_optimal_threshold(
            ensemble, 
            "col_ebma_ss", 
            ensemble["col_actual"], 
            df_cm_b, 
            df_cm_b,
            n_steps=1001
        )
        cm_ss_threshold_onset_1 = get_optimal_threshold(
            ensemble,
            "col_ebma_ss", 
            f"onset24_tx_{outcome}_1", 
            df_cm_b, 
            df_cm_b, 
            n_steps=1001
        )
        cm_ss_threshold_onset_5 = get_optimal_threshold(
            ensemble,
            "col_ebma_ss", 
            f"onset24_tx_{outcome}_5", 
            df_cm_b, 
            df_cm_b, 
            n_steps=1001
        )
        cm_ss_threshold_onset_25 = get_optimal_threshold(
            ensemble,
            "col_ebma_ss", 
            f"onset24_tx_{outcome}_25", 
            df_cm_b, 
            df_cm_b, 
            n_steps=1001
        )
        
    # Get the scores using these thresholds.
    ensemble["eval_ss"] = get_metrics(
        ensemble, 
        "col_ebma_ss", 
        ensemble["col_actual"], 
        df_cm_b,
        df_cm_b, 
        threshold=cm_ss_threshold, 
        brier_decimals=3
    )
    ensemble["eval_ss_onset_1"] = get_metrics(
        ensemble,
        "col_ebma_ss", 
        f"onset24_tx_{outcome}_1", 
        df_cm_b, 
        df_cm_b, 
        threshold=cm_ss_threshold_onset_1, 
        aupr_decimals=4,
        brier_decimals=3
    )
    ensemble["eval_ss_onset_5"] = get_metrics(
        ensemble,
        "col_ebma_ss", 
        f"onset24_tx_{outcome}_5", 
        df_cm_b, 
        df_cm_b, 
        threshold=cm_ss_threshold_onset_5,
        aupr_decimals=4,
        brier_decimals=3
    )
    ensemble["eval_ss_onset_25"] = get_metrics(
        ensemble,
        "col_ebma_ss", 
        f"onset24_tx_{outcome}_25", 
        df_cm_b, 
        df_cm_b, 
        threshold=cm_ss_threshold_onset_25, 
        aupr_decimals=4,
        brier_decimals=3
    )
    

In [ ]:
%%time
print("\n___ Getting the cm ensemble scores for sc... ___")
for ensemble in ensembles_cm:
    outcome = ensemble["outcome"]
    
    # Get the optimal thresholds (25brd here). Ensemble thresholds used for constituent model evaluation below.
    if "_1brd" not in ensemble["name"]:
        cm_sc_threshold = get_optimal_threshold(
            ensemble, 
            "col_ebma_sc", 
            ensemble["col_actual"], 
            df_cm_b, 
            df_cm_b, 
            n_steps=1001, 
            sc=True
        )
        cm_sc_threshold_onset_1 = get_optimal_threshold(
            ensemble,
            "col_ebma_sc", 
            f"onset24_tx_{outcome}_1", 
            df_cm_b, 
            df_cm_b, 
            n_steps=1001, 
            sc=True
        )
        cm_sc_threshold_onset_5 = get_optimal_threshold(
            ensemble,
            "col_ebma_sc", 
            f"onset24_tx_{outcome}_5", 
            df_cm_b, 
            df_cm_b, 
            n_steps=1001, 
            sc=True
        )
        cm_sc_threshold_onset_25 = get_optimal_threshold(
            ensemble,
            "col_ebma_sc", 
            f"onset24_tx_{outcome}_25", 
            df_cm_b, 
            df_cm_b, 
            n_steps=1001, 
            sc=True
        )
        
    # Get the scores using these thresholds.
    ensemble["eval_sc"] = get_metrics(
        ensemble, 
        "col_ebma_sc", 
        ensemble["col_actual"], 
        df_cm_b, 
        df_cm_b, 
        threshold=cm_sc_threshold, 
        sc=True,
        brier_decimals=3
    )
    ensemble["eval_sc_onset_1"] = get_metrics(
        ensemble,
        "col_ebma_sc", 
        f"onset24_tx_{outcome}_1", 
        df_cm_b, 
        df_cm_b, 
        threshold=cm_sc_threshold_onset_1, 
        sc=True,
        aupr_decimals=4,
        brier_decimals=3
    )
    ensemble["eval_sc_onset_5"] = get_metrics(
        ensemble,
        "col_ebma_sc", 
        f"onset24_tx_{outcome}_5", 
        df_cm_b, 
        df_cm_b, 
        threshold=cm_sc_threshold_onset_5, 
        sc=True,
        aupr_decimals=4,
        brier_decimals=3
    )
    ensemble["eval_sc_onset_25"] = get_metrics(
        ensemble,
        "col_ebma_sc", 
        f"onset24_tx_{outcome}_25", 
        df_cm_b, 
        df_cm_b, 
        threshold=cm_sc_threshold_onset_25, 
        sc=True,
        aupr_decimals=4,
        brier_decimals=3
    )

In [ ]:
%%time
print("___ Getting the pgm ensemble scores for ss... ___")
for ensemble in ensembles_pgm:
    outcome = ensemble["outcome"]
    
    # Get the optimal thresholds. Ensemble thresholds used for constituent model evaluation below.
    pgm_ss_threshold = get_optimal_threshold(
        ensemble, 
        "col_avg_ss", 
        ensemble["col_actual"], 
        df_pgm_b, 
        df_pgm_b,
        n_steps=11
    )
    pgm_ss_threshold_onset_1 = get_optimal_threshold(
        ensemble,
        "col_avg_ss", 
        f"onset24_tx_{outcome}_1", 
        df_pgm_b, 
        df_pgm_b, 
        n_steps=11
    )
    pgm_ss_threshold_onset_5 = get_optimal_threshold(
        ensemble,
        "col_avg_ss", 
        f"onset24_tx_{outcome}_5", 
        df_pgm_b, 
        df_pgm_b,
        n_steps=11
    )
    pgm_ss_threshold_onset_25 = get_optimal_threshold(
        ensemble,
        "col_avg_ss", 
        f"onset24_tx_{outcome}_25", 
        df_pgm_b, 
        df_pgm_b, 
        n_steps=11
    )

    # Get the scores using these thresholds.
    ensemble["eval_ss"] = get_metrics(
        ensemble, 
        "col_avg_ss", 
        ensemble["col_actual"], 
        df_pgm_b, 
        df_pgm_b, 
        threshold=pgm_ss_threshold
    ) 
    ensemble["eval_ss_onset_1"] = get_metrics(
        ensemble,
        "col_avg_ss",
        f"onset24_tx_{outcome}_1", 
        df_pgm_b, 
        df_pgm_b, 
        threshold=pgm_ss_threshold_onset_1,
        aupr_decimals=4
    )
    ensemble["eval_ss_onset_5"] = get_metrics(
        ensemble,
        "col_avg_ss", 
        f"onset24_tx_{outcome}_5", 
        df_pgm_b, 
        df_pgm_b, 
        threshold=pgm_ss_threshold_onset_5,
        aupr_decimals=4
    )
    ensemble["eval_ss_onset_25"] = get_metrics(
        ensemble,
        "col_avg_ss", 
        f"onset24_tx_{outcome}_25", 
        df_pgm_b, 
        df_pgm_b, 
        threshold=pgm_ss_threshold_onset_25,
        aupr_decimals=4
    ) 

In [ ]:
%%time
print("\n___ Getting the pgm ensemble scores for sc... ___")
for ensemble in ensembles_pgm:
    outcome = ensemble["outcome"]
    
    # Get the optimal thresholds. Ensemble thresholds used for constituent model evaluation below.
    pgm_sc_threshold = get_optimal_threshold(
        ensemble, 
        "col_avg_sc",
        ensemble["col_actual"], 
        df_pgm_b, 
        df_pgm_b,
        n_steps=11, 
        sc=True
    )  # Increase to 1001
    pgm_sc_threshold_onset_1 = get_optimal_threshold(
        ensemble,
        "col_avg_sc", 
        f"onset24_tx_{outcome}_1", 
        df_pgm_b, 
        df_pgm_b, 
        n_steps=11, 
        sc=True
    )
    pgm_threshold_onset_5 = get_optimal_threshold(
        ensemble,
        "col_avg_sc", 
        f"onset24_tx_{outcome}_5", 
        df_pgm_b, 
        df_pgm_b, 
        n_steps=11, 
        sc=True
    )
    pgm_threshold_onset_25 = get_optimal_threshold(
        ensemble,
        "col_avg_sc", 
        f"onset24_tx_{outcome}_25", 
        df_pgm_b, 
        df_pgm_b, 
        n_steps=11, 
        sc=True
    )
    
    # Get the scores using these thresholds.
    ensemble["eval_sc"] = get_metrics(
        ensemble, 
        "col_avg_sc", 
        ensemble["col_actual"], 
        df_pgm_b, 
        df_pgm_b,
        threshold=pgm_sc_threshold, 
        sc=True
    ) 
    ensemble["eval_sc_onset_1"] = get_metrics(
        ensemble,
        "col_avg_sc", 
        f"onset24_tx_{outcome}_1", 
        df_pgm_b, 
        df_pgm_b, 
        threshold=pgm_sc_threshold_onset_1, 
        sc=True,
        aupr_decimals=4
    )
    ensemble["eval_sc_onset_5"] = get_metrics(
        ensemble,
        "col_avg_sc", 
        f"onset24_tx_{outcome}_5", 
        df_pgm_b, 
        df_pgm_b, 
        threshold=pgm_threshold_onset_5, 
        sc=True,
        aupr_decimals=4
    )
    ensemble["eval_sc_onset_25"] = get_metrics(
        ensemble,
        "col_avg_sc", 
        f"onset24_tx_{outcome}_25", 
        df_pgm_b, 
        df_pgm_b, 
        threshold=pgm_threshold_onset_25, 
        sc=True,
        aupr_decimals=4
    )    

In [ ]:
%%time
print("___ Getting the cm constituent model scores for ss... ___")
for model in models_cm:
    outcome = model["outcome"]
    model["eval_ss"] = get_metrics(
        model, 
        "cols_ss", 
        model["col_actual"], 
        df_cm_b_calibrated, 
        df_cm_b, 
        threshold=cm_ss_threshold,
        brier_decimals=3
    )
    model["eval_ss_uncalibrated_a"] = get_metrics(
        model, 
        "cols_ss", 
        model["col_actual"], 
        df_cm_a, 
        df_cm_a, 
        threshold=cm_ss_threshold,
        brier_decimals=3
    )
    model["eval_ss_uncalibrated_b"] = get_metrics(
        model, 
        "cols_ss", 
        model["col_actual"], 
        df_cm_b, 
        df_cm_b, 
        threshold=cm_ss_threshold,
        brier_decimals=3
    )
    model["eval_ss_onset_1"] = get_metrics(
        model, 
        "cols_ss", 
        f"onset24_tx_{outcome}_1", 
        df_cm_b_calibrated, 
        df_cm_b, 
        threshold=cm_ss_threshold_onset_1,
        aupr_decimals=4,
        brier_decimals=3
    )
    model["eval_ss_onset_5"] = get_metrics(
        model, 
        "cols_ss", 
        f"onset24_tx_{outcome}_5", 
        df_cm_b_calibrated, 
        df_cm_b, 
        threshold=cm_ss_threshold_onset_5,
        aupr_decimals=4,
        brier_decimals=3
    )
    model["eval_ss_onset_25"] = get_metrics(
        model,
        "cols_ss", 
        f"onset24_tx_{outcome}_25", 
        df_cm_b_calibrated, 
        df_cm_b, 
        threshold=cm_ss_threshold_onset_25,
        aupr_decimals=4,
        brier_decimals=3
    )

In [ ]:
%%time
print("\n___ Getting the pgm constituent model scores for ss... ___")
for model in models_pgm:
    outcome = model["outcome"]
    model["eval_ss"] = get_metrics(
        model, 
        "cols_ss", 
        model["col_actual"], 
        df_pgm_b_calibrated, 
        df_pgm_b, 
        threshold=pgm_ss_threshold
    ) 
    model["eval_ss_uncalibrated_a"] = get_metrics(
        model, 
        "cols_ss",  
        model["col_actual"], 
        df_pgm_a,
        df_pgm_a,
        threshold=pgm_ss_threshold
    ) 
    model["eval_ss_uncalibrated_b"] = get_metrics(
        model, 
        "cols_ss", 
        model["col_actual"], 
        df_pgm_b, 
        df_pgm_b, 
        threshold=pgm_ss_threshold
    ) 
    model["eval_ss_onset_1"] = get_metrics(
        model, 
        "cols_ss",
        f"onset24_tx_{outcome}_1", 
        df_pgm_b_calibrated, 
        df_pgm_b, 
        threshold=pgm_ss_threshold_onset_1,
        aupr_decimals=4
    )
    model["eval_ss_onset_5"] = get_metrics(
        model, 
        "cols_ss", 
        f"onset24_tx_{outcome}_5", 
        df_pgm_b_calibrated, 
        df_pgm_b, 
        threshold=pgm_ss_threshold_onset_1,
        aupr_decimals=4
    )
    model["eval_ss_onset_25"] = get_metrics(
        model,
        "cols_ss",
        f"onset24_tx_{outcome}_25", 
        df_pgm_b_calibrated,
        df_pgm_b, 
        threshold=pgm_ss_threshold_onset_1,
        aupr_decimals=4
    )    

## E.2 Redo cm table II with MAL 

In [ ]:
# Add MAL scores to the model object.
steps = [1, 6, 12, 24, 36]
for ensemble in ensembles_cm:
    # For each step, get the ensemble's score, and
    # subtract that from the column in ablation_aupr.
    mal_aupr = ensemble["ablation_aupr"].copy()
    for step in steps:
        mal_aupr[step] = mal_aupr[step] - ensemble["eval_ss"][step]["AUPR"]
    ensemble["mal_aupr"] = mal_aupr
    
# Same for pgm.
for ensemble in ensembles_pgm:
    mal_aupr = ensemble["ablation_aupr"].copy()
    for step in steps:
        mal_aupr[step] = mal_aupr[step] - ensemble["eval_ss"][step]["AUPR"]
    ensemble["mal_aupr"] = mal_aupr

In [ ]:
# List the standard deviations for AUPR per ensemble_all step
# @TODO: incorporate bootstrapping code, for other metrics too.
sd_aupr = {
    "sb": {
        1: 0.0193,
        6: 0.0221,
        12: 0.0228,
        24: 0.0235,
        36: 0.0237
    },
    "ns": {
        1: 0.0434,
        6: 0.0390,
        12: 0.0432,
        24: 0.0443,
        36: 0.0451
    },
    "os": {
        1: 0.0487,
        6: 0.0458,
        12: 0.0465,
        24: 0.0505,
        36: 0.0478
    }
}

In [ ]:
%%time
steps_compare = [1, 36]
eval_keys = ["eval_ss"]

def modelname_without_outcome(model):
    return model["name"].replace(f"{model['outcome']}_", "")
    
    
def compare_models(models, level, steps, eval_key, constituent=True):
    """Compare models for a given set (constituent, ensemble). """
    ensemble_set = ensembles_cm if level=="cm" else ensembles_pgm
    
    # Get model names without the prefix.
    cd = {
        "model": [modelname_without_outcome(model) for model in models if model["outcome"] == "sb"]
    }
    
    # Add the scores per step.
    for step in steps:
        for outcome in ["sb", "ns", "os"]:
            models_this_outcome = [model for model in models if model["outcome"] == outcome]
            cd[f"aupr_{outcome}_{step}"] = [model[eval_key][step]["AUPR"] for model in models_this_outcome]
            
    # Separately add the MAL scores from ensembles.
    if constituent:
        for step in steps:
            for ensemble in ensemble_set:
                outcome = ensemble["outcome"]
                models_this_outcome = [model for model in models if model["outcome"] == outcome]
                if ensemble["name"] == f"{outcome}_ensemble_all":
                    mal_list = []
                    for modelname in [model["name"] for model in models_this_outcome]:
                        if modelname in ensemble["models"]:
                            mal_list.append(ensemble["mal_aupr"][step].loc[modelname].copy())
                        else:
                            mal_list.append(np.nan)
                    cd[f"mal_{outcome}_{step}"] = mal_list
    else:
        for step in steps:
            for ensemble in ensemble_set:
                outcome = ensemble["outcome"]
                if ensemble["name"] == f"{outcome}_ensemble_all":
                    cd[f"mal_{outcome}_{step}"] = np.nan

    return pd.DataFrame(cd)

In [ ]:
for eval_key in eval_keys:
    # Set paths according to eval key.
    if eval_key=="eval_ss":
        out = dir_tables_eval_constituent_compare_incidence
    else:
        out = dir_tables_eval_constituent_compare_onset

    # cm-level
    cm_const_in = compare_models(
        [model for model in models_cm],
        "cm",
        steps_compare,
        eval_key
    )
    cm_ens_in = compare_models(
        [model for model in ensembles_cm if not model["name"].endswith("_lbrd")], 
        "cm",
        steps_compare,
        eval_key,
        constituent=False
    )
    # Append cm ensemble scores to constituent model scores.
    cm_in = cm_const_in.append(cm_ens_in, ignore_index=True)
    
    # pgm-level
    pgm_const_in = compare_models(
        [model for model in models_pgm], 
        "pgm",
        steps_compare,
        eval_key
    )
    pgm_ens_in = compare_models(
        [model for model in ensembles_pgm], 
        "pgm",
        steps_compare,
        eval_key,
        constituent=False
    )
    # Append pgm ensemble scores to constituent model scores.
    pgm_in = pgm_const_in.append(pgm_ens_in, ignore_index=True)
    
    # Round the scores.
    for data in [cm_in, pgm_in]:
        for col in [col for col in data.columns if "aupr" in col]:
            data[col] = round(data[col], 3)
        for col in [col for col in data.columns if "mal" in col]:
            data[col] = round(data[col], 4)

In [ ]:
# Small fix of column order.
columns_reordered = ["model"]
for outcome in ["sb", "ns", "os"]:
    for step in steps_compare:
        columns_reordered.append(f"aupr_{outcome}_{step}")
        columns_reordered.append(f"mal_{outcome}_{step}")
cm_in = cm_in[columns_reordered]
pgm_in = pgm_in[columns_reordered]

In [ ]:
# Write .tex for cm.
# @TODO(Remco): fix pathsetting in eval_tex to os.path.join.
eval_tex.write_eval_mal_table(
    cm_in, 
    dir_tables_eval_constituent_compare_incidence + "/",
    f"cm_outcome_twostep_1_36_comparison"
)
# Write .tex for pgm.
eval_tex.write_eval_mal_table(
    pgm_in, 
    dir_tables_eval_constituent_compare_incidence + "/",
    f"pgm_outcome_twostep_1_36_comparison"
)

In [ ]:
# SEPARATE DATAPREP FOR ABLATION FIGURES (CM)

# Add AUPR standard deviations derived from bootstraps.
fig_out = cm_const_in.copy()
for outcome in ["sb", "ns", "os"]:
    for step in [1, 6, 36]:
        fig_out[f"sd_aupr_{outcome}_{step}"] = sd_aupr[outcome][step]

In [ ]:
# Add model weights.
for ensemble in ensembles_cm:
    if "_all" in ensemble["name"]:
        outcome = ensemble["outcome"]
        for step, column in ensemble["weights_ebma_bc"].items():
            fig_out[f"weight_{outcome}_{step}"] = [score for item, score in column.items()]

In [ ]:
fig_out.to_csv(os.path.join(dir_tables, "ablation_loss.csv"), index=False)

## Figure 3 - evaluating models' performance in ensemble

In [ ]:
#read data
df =  pd.read_csv(os.path.join(dir_cache, 'ablation_loss_ens.csv'),sep=";")
plt.rcParams.update({'font.size': 32})

def ablation_plot(df, outc, invert_mal = True):
    
    # Make a copy of df so we can mess with it fearlessly
    df = df.copy()
    
    steps = [1, 6, 36]
    n_steps = len(steps)
    
    if invert_mal:
        for col_mal in [col for col in df if col.startswith("mal_")]:
            df[col_mal] = -df[col_mal]
    
    cols_keep = ["model"]
    for step in steps:
        for col in [
            f"aupr_ens_{outc}_{step}", 
            f"aupr_{outc}_{step}", 
            f"mal_{outc}_{step}",
            f"weight_{outc}_{step}",
            f"sd_aupr_{outc}_{step}",
        ]:
            cols_keep.append(col)
    df = df[cols_keep]
    
    fig, (axes) = plt.subplots(1, n_steps)
    fig.set_size_inches(10*n_steps, 12)

    # Write models as the Y axis label
    fig.text(-0.09, 0.5, "Models", va='center', rotation='vertical', fontsize=35)
    
    # Set tight layout
    fig.tight_layout(pad=0.6)
    plt.subplots_adjust(wspace=0, hspace=0)

    circle_size = 16000

    for step, ax in zip(steps, axes):
        col_mal = f"mal_{outc}_{step}"
        col_weight = f"weight_{outc}_{step}"
        col_aupr = f"aupr_{outc}_{step}"
        # Draw the circles
        scatter = ax.scatter(
                df[col_mal],
                df['model'],
                s=df[col_weight]*circle_size,
                c=df[col_aupr], 
                cmap = 'rainbow', 
                vmin=0, 
                vmax=1,
                alpha = 1,
                marker='o'
        )
        
        # Drop model names from axes other than s=1
        if step > 1:
            ax.set_yticklabels([])
        
        # left - right lines
        ax.grid(color='silver', linestyle='-.', linewidth=0.7, axis = 'y')
        
        # Plot gray field
        # Mid should be zero right becausue MAL of the ensemble is zero by definition?
        aupr_ens_mal = 0 
        half_sd_aupr_step = df[f"sd_aupr_{outc}_{step}"].loc[0] * 0.5
        gray_left = aupr_ens_mal - half_sd_aupr_step
        gray_right = aupr_ens_mal + half_sd_aupr_step
        ax.axvline(aupr_ens_mal, color='silver', linestyle='--', linewidth = 1)
        ax.axvline(gray_left, color='silver', alpha = 0.5, linestyle='--', linewidth = 1)
        ax.axvline(gray_right, color='silver', linestyle='--', linewidth = 1)
        ax.axvspan(gray_left, gray_right, alpha=0.2, color='silver')
        
        # Ax title
        ax.title.set_text(f"S = {step}")
        ax.set_xlabel("Change in AUPR")
        
        # Limit the x_axis
        x_offset = 0.03
        x_min, x_max = aupr_ens_mal - x_offset, aupr_ens_mal + x_offset
        ax.set_xlim([x_min, x_max])
        
        # Label the x axis
        #x_labels = np.linspace(x_min, x_max)
        #x_labels = [round(x_label, 3) for x_label in x_labels]
        #ax.set_xticklabels(x_labels)
        ax.set_xticks(ax.get_xticks()[:-1])
        ax.set_xticks(ax.get_xticks()[1:])
     
   #Add colorbar
    im = plt.gca().get_children()[0]
    cax = fig.add_axes([1,0,0.03,0.97]) 
    fig.colorbar(im, cax).set_label('Constituent model AUPR', horizontalalignment='center')
    
    # Add model weight legend
    handles, labels= scatter.legend_elements(prop="sizes", alpha = 0.6)
    weight_legend_values = ["0-0.039","0.06-0.079", "0.1-0.11","0.16-0.17","0.24-0.25",">0.26"]
    legend = fig.legend(
        handles, 
        weight_legend_values, 
        loc="center right",
        bbox_to_anchor=(1.3, 0, 0, 1),
        #borderaxespad=1.5,
        #borderpad = -5,
        labelspacing=1.2,
        #handletextpad = 3,
        frameon=False,
        title="Model Weight",
        
    )

ablation_plot(df, outc="sb")
ablation_plot(df, outc="ns")
ablation_plot(df, outc="os")

## E.3 Redo appendix D step-comparison tables

In [ ]:
%%time

step_sets = ([1, 6], [12, 36])
eval_keys = ["eval_ss"]
paramdict = {
    "constituent": {
        "incidence": dir_tables_eval_constituent_compare_incidence,
        "onset": dir_tables_eval_constituent_compare_onset,
        "cm": models_cm,
        "pgm": models_pgm
    },
    "ensemble": {
        "incidence": dir_tables_eval_ensembles_compare_incidence,
        "onset": dir_tables_eval_ensembles_compare_onset,
        "cm": ensembles_cm,
        "pgm": ensembles_pgm
    }
}

for steps in step_sets:
    for modelset in ["constituent"]:
        for level in ["cm", "pgm"]:
            for eval_key in eval_keys:
                if eval_key=="eval_ss":
                    out = paramdict[modelset]["incidence"] 
                else:
                    out= paramdict[modelset]["onset"]
                for outc in ["sb", "ns", "os"]:
                    for step in steps:
                        # Get scores per model.
                        models = paramdict[modelset][level]
                        scores = []
                        for model in models:
                            if model["outcome"]==outc:
                                rd = model[eval_key][step].copy()
                                rd["Model"] = model["name"]
                                # Add MAL separately. MAL is missing for some pgm models given ensemble specs.
                                try:
                                    for ensemble in paramdict["ensemble"][level]:
                                        if ensemble["name"] == f"{outc}_ensemble_all":
                                            rd["MAL"] = ensemble["mal_aupr"][step].loc[model["name"]].copy()
                                except Exception:
                                    print("No MAL for {}.".format(model["name"]))
                                    for ensemble in paramdict["ensemble"][level]:
                                        if ensemble["name"] == f"{outc}_ensemble_all":
                                            rd["MAL"] = None
                                scores.append(rd)
                        models_scores = pd.DataFrame(
                            scores, 
                            columns=[
                                "Model", 
                                "AUROC", 
                                "AUPR", 
                                "Brier",
                                "MAL"
                            ]
                        )
                        # Rounding correction of MAL.
                        models_scores["MAL"] = round(models_scores["MAL"], 4)
                        
                        # Rename columns per step.
                        models_scores.columns = [
                            "Model", 
                            f"{step}_AUROC", 
                            f"{step}_AUPR", 
                            f"{step}_Brier",
                            f"{step}_MAL",
                        ]
                        if step == steps[0]:
                            df = models_scores 
                        else:
                            df = df.merge(models_scores, on="Model")

                    step_one = steps[0]
                    step_two = steps[1]
                    
                    # Fix decimals.
                    for col in [col for col in df.columns if "AUPR" in col]:
                        df[col] = round(df[col], 3)
                    for col in [col for col in df.columns if "AUROC" in col]:
                        df[col] = round(df[col], 3)
                    for col in [col for col in df.columns if "Brier" in col]:
                        decimals = 4 if level == "pgm" else 3
                        df[col] = round(df[col], decimals)
                    for col in [col for col in df.columns if "MAL" in col]:
                        df[col] = round(df[col], 4)
                    
                    eval_tex.write_outcome_step_comparison(
                        df=df, 
                        to_path=paramdict[modelset]["incidence"] + "/",
                        filename=f"{level}_twostep_{step_one}_{step_two}_comparison_{outc}",
                        steps=steps,
                        add_mal=True
                    ) 

## E.4 Individual model-set tables
    

In [ ]:
%%time
# Parameters.
do_individual = True
eval_keys = ["eval_ss", 
             "eval_ss_onset_1", 
             "eval_ss_onset_5", 
             "eval_ss_onset_25"]

# Evaluation scores (note confusion matrix set to false).
def write_eval_tables(step, level, outc, models, eval_key, path, confusion=False):
    """ Write a latex table with evaluation metrics"""
    
    # Collect scores per model for outc-step.
    scores = []
    for model in models:
        if model["outcome"]==outc:
            rd = {"Model": model["name"]}
            rd.update(model[eval_key][step].copy())
            scores.append(rd)
            
    df = pd.DataFrame(scores)
    
    # Fix decimals.
    for col in [col for col in df.columns if "AUPR" in col]:
        df[col] = round(df[col], 3)
    for col in [col for col in df.columns if "AUROC" in col]:
        df[col] = round(df[col], 3)
    for col in [col for col in df.columns if "Brier" in col]:
        decimals = 4 if level == "pgm" else 3
        df[col] = round(df[col], decimals)
    
    eval_tex.write_eval_table(
        scores=df,
        to_path=f"{os.path.dirname(path)}/",
        filename=os.path.basename(path) + f"_{eval_key}_{outc}_s{step}",
        confusion=confusion
    )
    
    return df

In [ ]:
if do_individual: 
    for level in ["cm", "pgm"]:
        for step in [1, 6, 12, 24, 36]:
            for outc in ["sb", "ns", "os"]:
                for eval_key in eval_keys:
                    if eval_key=="eval_ss":
                        const_out = dir_tables_eval_constituent_incidence
                        ens_out = dir_tables_eval_ensembles_incidence
                    else:
                        const_out = dir_tables_eval_constituent_onset
                        ens_out = dir_tables_eval_ensembles_onset
                    models = models_cm if level=="cm" else models_pgm
                    ensembles = ensembles_cm if level=="cm" else ensembles_pgm
                    # Write tables for the constituent models.
                    write_eval_tables(
                        step,
                        level,
                        outc,
                        models,
                        eval_key,
                        os.path.join(const_out, f"scores_constituent_{level}_{step}")
                    )
                    # Write tables for the ensembles.
                    write_eval_tables(
                        step,
                        level,
                        outc,
                        ensembles,
                        eval_key,
                        os.path.join(ens_out, f"scores_ensembles_{level}_{step}")
                    )
                    

## E.5 Ensemble comparison tables (new vs. old) 

In [ ]:
%%time
# NOTE: Ensemble comparison on sc here only, given that the 'old' is sc-only.
# Parameters.
secondary_metric = "Brier"  # alternative: "AUROC"
run_id = "r_2020_02_01"
eval_keys = ["eval_sc", 
             "eval_sc_onset_1", 
             "eval_sc_onset_5", 
             "eval_sc_onset_25"]

# Setup for within-function string-formatting.
cols_onset = ["onset24_tx_{{outcome}}_{i}".format(i=i) for i in [1, 5, 25]]  
actual_keys = ["ged_dummy_{outcome}"] + cols_onset

# Get results.
for level in ["cm", "pgm"]:
    for eval_key, actual_key in zip(eval_keys, actual_keys):
        # Set up parameters.
        df_actual = df_cm_b if level=="cm" else df_pgm_b
        models = ensembles_cm if level=="cm" else ensembles_pgm
        prediction_key = "col_ebma_sc" if level =="cm" else "col_avg_sc"

        # Get the old ensemble results (note the separate function).
        old_ens = eval_data.collect_prev_scores(
            run_id=run_id, 
            df_actual=df_actual, 
            actual_key=actual_key, 
            level=level, 
            secondary=secondary_metric.lower()
        )
        old_ens.columns = [
            "Model", 
            "Outcome", 
            "Previous AUPR", 
            f"Previous {secondary_metric}"
        ]

        # Get the new ensemble results.
        new_scores = []
        for model in models:
            rd = model[eval_key].copy()
            rd["Model"] = model["name"]
            rd["Outcome"] = model["outcome"]
            new_scores.append(rd)
        new_ens = pd.DataFrame(
            new_scores, 
            columns=["Model", "Outcome", "AUPR", f"{secondary_metric}"]
        )

        # If cm, only take the 25brd for onset, 1brd otherwise. 
        if level=="cm" and eval_key=="eval_sc":
            new_ens = new_ens[3:]
        elif level=="cm" and eval_key!="eval_sc":
            new_ens = new_ens[:3]
        
        # Reset and rename columns for a clear merge.
        new_ens = new_ens[["Outcome", "AUPR", f"{secondary_metric}"]]
        new_ens.columns = ["Outcome", "New AUPR", f"New {secondary_metric}"]
    
        # Merge and drop outcome col.
        df = old_ens.merge(new_ens, on="Outcome")
        df.drop(columns=['Outcome'], inplace=True)

        # Reorder and write to .tex.
        df = df[["Model", "New AUPR", "Previous AUPR", 
                 f"New {secondary_metric}", f"Previous {secondary_metric}"]]
        
        # Fix decimals.
        for col in [col for col in df.columns if "AUPR" in col]:
            df[col] = round(df[col], 3)
        for col in [col for col in df.columns if "AUROC" in col]:
            df[col] = round(df[col], 3)
        for col in [col for col in df.columns if "Brier" in col]:
            decimals = 4 if level == "pgm" else 3
            df[col] = round(df[col], decimals)
                        
        eval_tex.write_ensemble_comparison(
            df, 
            os.path.join(dir_tables_eval_ensembles_oldvsnew, f"ens_{level}_{eval_key}_comparison.tex"),
            secondary=secondary_metric.lower()
        )
        

## E.6 Ensemble comparison tables (EBMA vs. weighted)

In [ ]:
%%time
secondary_metric = "Brier"  # alternative: "AUROC"

df = eval_data.compare_ebma(
        models=[model for model in ensembles_cm if not model["name"].endswith("_1brd")],
        df_pred=df_cm_b,
        df_actual=df_cm_b,
        ebma_key="col_ebma_ss",
        avg_key="col_avg_ss",
        secondary=secondary_metric.lower()
    )

# Fix decimals.
for col in [col for col in df.columns if "AUPR" in col]:
    df[col] = round(df[col], 3)
for col in [col for col in df.columns if "AUROC" in col]:
    df[col] = round(df[col], 3)
for col in [col for col in df.columns if "Brier" in col]:
    decimals = 4 if level == "pgm" else 3
    df[col] = round(df[col], decimals)

eval_tex.write_ebma_comparison(
    df=df,
    to_path=os.path.join(dir_tables_eval_ensembles_ebmavsunweighted, "ebma_comparison.tex"),
    secondary=secondary_metric.lower()
)

## E.7 Onset vs incidence comparison tables

In [ ]:
%%time
# NOTE: currently ss only.
secondary_metric = "Brier"  # alternative: "AUROC"
onset_keys = ["eval_ss_onset_1", 
             "eval_ss_onset_5", 
             "eval_ss_onset_25"]
paramdict = {
    "constituent": {
        "out": dir_tables_eval_constituent_incidencevsonset,
        "cm": models_cm,
        "pgm": models_pgm
        },
    "ensemble": {
        "out": dir_tables_eval_ensembles_incidencevsonset,
        "cm": ensembles_cm,
        "pgm": ensembles_pgm
        }
    }

for modelset in ["constituent", "ensemble"]:
    for level in ["cm", "pgm"]:
        # Specify model set and output dirs.
        models = paramdict[modelset][level]
        out = paramdict[modelset]["out"]
        for onset_key in onset_keys:
            incidence_scores = []
            onset_scores = []
            for model in models:
                for step in [1, 6, 12, 24, 36]:
                # Get the incidence scores.
                    rd = model["eval_ss"][step].copy()
                    rd["model"] = model["name"]
                    rd["outcome"] = model["col_actual"]
                    rd["step"] = step
                    incidence_scores.append(rd)
                # Get the onset scores.
                    rd = model[onset_key][step].copy()
                    rd["model"] = model["name"]
                    rd["outcome"] = model["col_actual"]
                    rd["step"] = step
                    onset_scores.append(rd)
            # Incidence df.
            incidence_df = pd.DataFrame(
                incidence_scores, 
                columns=[
                    "model", 
                    "outcome", 
                    "step", 
                    "AUPR", 
                    f"{secondary_metric}"
                ]
            )
            incidence_df.columns = [
                "model", 
                "outcome", 
                "step",
                "incidence_aupr", 
                f"incidence_{secondary_metric.lower()}"
            ]
            # Onset df.
            onset_df = pd.DataFrame(
                onset_scores, 
                columns=[
                    "model", 
                    "outcome", 
                    "step", 
                    "AUPR", 
                    f"{secondary_metric}"
                ]
            )
            onset_df.columns = [
                "model", 
                "outcome", 
                "step",
                "onset_aupr", 
                f"onset_{secondary_metric.lower()}"
            ]
            # Merge and reorder the columns.
            output = incidence_df.merge(onset_df, on=["model", "outcome", "step"])
            output = output[["model", "outcome", "step", "incidence_aupr",
                       "onset_aupr", f"incidence_{secondary_metric.lower()}", 
                       f"onset_{secondary_metric.lower()}"]]
            
            # Fix decimals.
            for col in [col for col in output.columns if "aupr" in col]:
                output[col] = round(output[col], 3)
            for col in [col for col in output.columns if "auroc" in col]:
                output[col] = round(output[col], 3)
            for col in [col for col in output.columns if "brier" in col]:
                decimals = 4 if level == "pgm" else 3
                output[col] = round(output[col], decimals)
            
            # Write table to file.
            eval_tex.write_onset_comparison(
                df=output,
                to_path=os.path.join(out, 
                                     f"{level}_{modelset}_{onset_key}_comparison.tex"),
                secondary=secondary_metric.lower(),
            )
            

## E.8 Comparison tables: all metrics according to selected steps, by outcome
Disable for now given preference for the more condensed comparison table (E.7) below.

In [ ]:
%%time
do_comparison_outcome = False

steps = [1, 6, 36]
eval_keys = ["eval_ss", 
             "eval_ss_onset_1", 
             "eval_ss_onset_5", 
             "eval_ss_onset_25"]
paramdict = {
    "constituent": {
        "incidence": dir_tables_eval_constituent_compare_incidence,
        "onset": dir_tables_eval_constituent_compare_onset,
        "cm": models_cm,
        "pgm": models_pgm
        },
    "ensemble": {
        "incidence": dir_tables_eval_ensembles_compare_incidence,
        "onset": dir_tables_eval_ensembles_compare_onset,
        "cm": ensembles_cm,
        "pgm": ensembles_pgm
        }
    }

if do_comparison_outcome:
    for modelset in ["constituent", "ensemble"]:
        for level in ["cm", "pgm"]:
            for eval_key in eval_keys:
                if eval_key=="eval_ss":
                    out = paramdict[modelset]["incidence"] 
                else:
                    out= paramdict[modelset]["onset"]
                for outc in ["sb", "ns", "os"]:
                    for step in steps:
                        # Get scores per model.
                        models = paramdict[modelset][level]
                        scores = []
                        for model in models:
                            if model["outcome"]==outc:
                                rd = model[eval_key][step].copy()
                                rd["Model"] = model["name"]
                                scores.append(rd)
                        models_scores = pd.DataFrame(
                            scores, 
                            columns=[
                                "Model", 
                                "AUROC", 
                                "AUPR", 
                                "Brier"
                            ]
                        )
                        # Rename columns per step.
                        models_scores.columns = [
                            "Model", 
                            f"{step}_AUROC", 
                            f"{step}_AUPR", 
                            f"{step}_Brier"
                        ]
                        if step == steps[0]:
                            df = models_scores 
                        else:
                            df = df.merge(models_scores, on="Model")
                            
                    # Fix decimals.
                    for col in [col for col in df.columns if "AUPR" in col]:
                        df[col] = round(df[col], 3)
                    for col in [col for col in df.columns if "AUROC" in col]:
                        df[col] = round(df[col], 3)
                    for col in [col for col in df.columns if "Brier" in col]:
                        decimals = 4 if level == "pgm" else 3
                        df[col] = round(df[col], decimals)

                    # Write comparison to file.
                    eval_tex.write_outcome_step_comparison(
                        df, 
                        out,
                        f"/comparison_{modelset}_{level}_{eval_key}_{outc}", 
                        steps=steps
                    )   
                    

## E.9 Comparison tables: all outcomes by selected steps, single or dual metrics

In [ ]:
%%time
do_comparison_all = True

steps_compare = [1, 36]
eval_keys = ["eval_ss", 
             "eval_ss_onset_1", 
             "eval_ss_onset_5", 
             "eval_ss_onset_25"]

def compare_models(models, steps, eval_key, brier=True):
    def modelname_without_outcome(model):
        return model["name"].replace(f"{model['outcome']}_", "")

    cd = {
        "Model": [modelname_without_outcome(model) for model in models if model["outcome"] == "sb"]
    }
    for step in steps:
        for outcome in ["sb", "ns", "os"]:
            models_this_outcome = [model for model in models if model["outcome"] == outcome]
            cd[f"{outcome}_aupr_{step}"] = [model[eval_key][step]["AUPR"] for model in models_this_outcome]

    # Separately for the Brier scores.
    if brier:             
        for step in steps:
            for outcome in ["sb", "ns", "os"]:
                models_this_outcome = [model for model in models if model["outcome"] == outcome]
                cd[f"{outcome}_brier_{step}"] = [model[eval_key][step]["Brier"] for model in models_this_outcome]
                                     
    return pd.DataFrame(cd)
                                     
if do_comparison_all:
    for eval_key in eval_keys:
        # Set paths according to eval key.
        if eval_key=="eval_ss":
            out = dir_tables_eval_constituent_compare_incidence
        else:
            out = dir_tables_eval_constituent_compare_onset
    
        # Append cm ensemble scores to constituent model scores.
        cm_ens_in = compare_models(
            [model for model in ensembles_cm if not model["name"].endswith("_lbrd")], 
            steps_compare,
            eval_key
        )
        cm_const_in = compare_models(
            [model for model in models_cm], 
            steps_compare,
            eval_key
        )
        cm_in = cm_const_in.append(cm_ens_in)
        
        # Round the scores.
        for col in [col for col in cm_in.columns if "aupr" in col]:
            cm_in[col] = round(cm_in[col], 3)
        for col in [col for col in cm_in.columns if "brier" in col]:
            cm_in[col] = round(cm_in[col], 4)
        
        # Write table.
        eval_tex.write_step_comparison(
            df=cm_in,
            to_path=f"{out}/",
            steps=steps_compare,
            filename=f"outcome_{eval_key}_twostep_cm",
            add_metric="Brier"  # If without, remove and set brier to False in compare_models.
        )
           
        # Append pgm ensemble scores to constituent model scores.
        pgm_ens_in = compare_models(
            [model for model in ensembles_pgm], 
            steps_compare,
            eval_key
        )
        pgm_const_in = compare_models(
            [model for model in models_pgm], 
            steps_compare,
            eval_key
        )
        pgm_in = pgm_const_in.append(pgm_ens_in)
        
        # Round the scores.
        for col in [col for col in pgm_in.columns if "aupr" in col]:
            pgm_in[col] = round(pgm_in[col], 3)
        for col in [col for col in pgm_in.columns if "brier" in col]:
            pgm_in[col] = round(pgm_in[col], 4)

        # Write table.
        eval_tex.write_step_comparison(
            df=pgm_in,
            to_path=f"{out}/",
            steps=steps_compare,
            filename=f"outcome_{eval_key}_twostep_pgm",
            add_metric="Brier"
        )

## DIAGNOSTIC FIGURES

In [ ]:
%%time

do_pgm = False
do_ss = True

pgm_bounds = [-5.32, 11.47, 2.21, 17.21]
select_priogrid = eval_data.fetch_priogrid(bounds=pgm_bounds)

# Ensembles, cm sc.
for ensemble in ensembles_cm:
    # Prepare params and current data.
    col_sc = ensemble["col_ebma_sc"]
    col_actual = ensemble["col_actual"]
    outcome = ensemble["outcome"]
    cols_onset = [f"onset24_tx_{outcome}_{i}" for i in [1, 5, 25]]
    current = eval_data.fetch_ensemble_predictions(
        run_id=run_id, level="cm", outcome=outcome, eval_test=True
    )
    col_current = current.columns[0]
    actuals = df_cm_b[col_actual]
    onsets = df_cm_b[cols_onset]
    probs = df_cm_b[col_sc]
    # Merge and add iso_date.
    df = current.merge(probs, left_index=True, right_index=True)
    df = df.merge(actuals, left_index=True, right_index=True)
    df = df.merge(onsets, left_index=True, right_index=True)
    df = eval_data.add_iso_date(df)
    # Generate plots.
    # NOTE: Only for 1brd here.
    filename = col_sc.replace(".", "_")
    if "ged_dummy" in ensemble["col_actual"]:
        eval_plot.bisep_plot(
            df=df,
            model_x=col_current,
            model_y=col_sc,
            outcome=col_actual,
            label="iso_date",
            title=f"New and old ensembles, {outcome} (2016-2018)",
            filename=f"bisep_{filename}",
            path_out=f"{dir_plots_diagnostic_cm}/",
        )
        eval_plot.mc_plot(
            df=df,
            model=col_sc,
            outcome=col_actual,
            label="iso_date",
            title=f"New ensemble sc on {outcome} (2016-2018)",
            filename=f"mcplot_{filename}",
            path_out=f"{dir_plots_diagnostic_cm}/",
        )
        # Also generate the mc plot for the old ensemble.
        eval_plot.mc_plot(
            df=df,
            model=col_current,
            outcome=col_actual,
            label="iso_date",
            title=f"Old ensemble sc on {outcome} (2016-2018)",
            filename=f"mcplot_{col_current}",
            path_out=f"{dir_plots_diagnostic_cm}/",
        )
    else:
        # Generate the plots for the onset outcomes.
        # NOTE: 25 brd outcome here.
        for onset in cols_onset:
            eval_plot.bisep_plot(
                df=df,
                model_x=col_current,
                model_y=col_sc,
                outcome=onset,
                label="iso_date",
                title=f"New and old ensembles, {onset} (2016-2018)",
                filename=f"bisep_{onset}_{filename}",
                path_out=f"{dir_plots_diagnostic_cm}/",
            )
            eval_plot.mc_plot(
                df=df,
                model=col_sc,
                outcome=onset,
                label="iso_date",
                title=f"New ensemble sc on {onset} (2016-2018)",
                filename=f"mcplot_{onset}_{filename}",
                path_out=f"{dir_plots_diagnostic_cm}/",
            )
            eval_plot.mc_plot(
                df=df,
                model=col_current,
                outcome=onset,
                label="iso_date",
                title=f"Old ensemble sc on {onset} (2016-2018)",
                filename=f"mcplot_{onset}_{col_current}",
                path_out=f"{dir_plots_diagnostic_cm}/",
            )

# Second, generate plots per ss.
# NOTE: Only 25brd in this case.
if do_ss:
    for ensemble in ensembles_cm:
        if "ged_dummy" not in ensemble["col_actual"]:
            for step in ensemble["steps"]:
                # Prepare params and current data.
                col_ss = ensemble["col_ebma_ss"][step]
                col_actual = ensemble["col_actual"]
                outcome = ensemble["outcome"]
                cols_onset = [f"onset24_tx_{outcome}_{i}" for i in [1, 5, 25]]
                current = eval_data.fetch_ensemble_predictions(
                    run_id=run_id, level="cm", outcome=outcome, eval_test=True
                )
                col_current = current.columns[0]
                # Slice by step.
                # step = 1 is index[0] as month_index is a zero-indexed list
                month_index = df_cm_b.index.get_level_values(0).unique()[step - 1] 
                actuals = df_cm_b.loc[slice(month_index, month_index), :][col_actual]
                onsets = df_cm_b.loc[slice(month_index, month_index), :][cols_onset]
                probs = df_cm_b.loc[slice(month_index, month_index), :][col_ss]
                current = current.loc[slice(month_index, month_index), :]
                # Merge and add iso-date.
                df = current.merge(probs, left_index=True, right_index=True)
                df = df.merge(actuals, left_index=True, right_index=True)
                df = df.merge(onsets, left_index=True, right_index=True)
                df = eval_data.add_iso_date(df)
                # Plots.
                filename = col_ss.replace(".", "_")
                eval_plot.bisep_plot(
                    df=df,
                    model_x=col_current,
                    model_y=col_ss,
                    outcome=col_actual,
                    label="iso_date",
                    title=f"New and old ensembles, {outcome} s{step}",
                    filename=f"bisep_{filename}",
                    path_out=f"{dir_plots_diagnostic_cm}/",
                )
                eval_plot.mc_plot(
                    df=df,
                    model=col_ss,
                    outcome=col_actual,
                    label="iso_date",
                    title=f"{col_ss} at s{step}",
                    filename=f"mcplot_{filename}",
                    path_out=f"{dir_plots_diagnostic_cm}/",
                )
                # For the onset outcomes.
                for onset in cols_onset:
                    eval_plot.bisep_plot(
                        df=df,
                        model_x=col_current,
                        model_y=col_ss,
                        outcome=onset,
                        label="iso_date",
                        title=f"New and old ensembles, {onset} s{step}",
                        filename=f"bisep_{onset}_{filename}",
                        path_out=f"{dir_plots_diagnostic_cm}/",
                    )
                    eval_plot.mc_plot(
                        df=df,
                        model=col_ss,
                        outcome=onset,
                        label="iso_date",
                        title=f"{col_ss} at s{step} on {onset}",
                        filename=f"mcplot_{onset}_{filename}",
                        path_out=f"{dir_plots_diagnostic_cm}/",
                    )


if do_pgm:
    for ensemble in ensembles_pgm:
        for step in ensemble["steps"]:
            # Prepare params and current data.
            col_ss = ensemble["col_avg_ss"][step]  # ebma not found in df_pgm_b?
            col_actual = ensemble["col_actual"]
            outcome = ensemble["outcome"]
            current = eval_data.fetch_ensemble_predictions(
                run_id=run_id, level="pgm", outcome=outcome, eval_test=True
            )
            col_current = current.columns[0]
            # Slice by step.
            # step = 1 is index [0] as month_index is a zero-indexed list
            month_index = df_pgm_b.index.get_level_values(0).unique()[step - 1]
            actuals = df_pgm_b.loc[slice(month_index, month_index), :][col_actual]
            probs = df_pgm_b.loc[slice(month_index, month_index), :][col_ss]
            current = current.loc[slice(month_index, month_index), :]
            # Isolate priogrids of interest.
            current.reset_index(inplace=True)
            df = select_priogrid.merge(current, on="pg_id")
            # Merge and add coord_date.
            df.set_index(["month_id", "pg_id"], inplace=True)
            df = df.merge(probs, left_index=True, right_index=True)
            df = df.merge(actuals, left_index=True, right_index=True)
            df = eval_data.add_iso_date(df)
            df["iso_coord_date"] = (
                "("
                + df["latitude"].map(str)
                + ", "
                + df["longitude"].map(str)
                + ") "
                + df["month_str"]
            )
            # Plots.
            filename = col_ss.replace(".", "_")
            eval_plot.mc_plot(
                df=df,
                model=col_ss,
                outcome=col_actual,
                label="iso_coord_date",
                title=f"Model diagnostics, {col_ss} at s{step}",
                filename=f"mcplot_{filename}",
                path_out=f"{dir_plots_diagnostic_pgm}/",
            )
            eval_plot.bisep_plot(
                df=df,
                model_x=df.columns[5],
                model_y=col_ss,
                outcome=col_actual,
                label="iso_coord_date",
                title=f"Biseparation of new and old ensembles, {outcome} s{step}",
                filename=f"bisep_{filename}",
                path_out=f"{dir_plots_diagnostic_pgm}/",
            )

## HEATMAPS

In [ ]:
#@TODO: Use os.path.join to make paths, not strings and trailing /
for ensemble in ensembles_cm:
    for step in ensemble["steps"]:
        # Prepare params and data.
        col_ss = ensemble["col_ebma_ss"][step]
        country = eval_data.fetch_country()
        probs = pd.DataFrame(df_cm_b[col_ss])
        probs.reset_index(inplace=True)
        df = probs.merge(country, on="country_id")
        # Note that the function works with unindexed data.
        eval_plot.plot_heatmap(
            df=df,
            colname_feature=col_ss,
            level="cm",
            variable_scale="prob",
            to_path=f"{dir_plots_heatmaps_cm}/",
            run="r_2020_02_01",
        )

priogrid = eval_data.fetch_priogrid()
for ensemble in ensembles_pgm:
    for step in ensemble["steps"]:
        # Prepare params and data.
        col_ss = ensemble["col_avg_ss"][step]
        country = eval_data.fetch_country()
        probs = pd.DataFrame(df_pgm_b[col_ss])
        probs.reset_index(inplace=True)
        probs = probs.merge(priogrid, on="pg_id")
        df = probs.merge(country, on="country_id")
        # Note that the function works with unindexed data.
        eval_plot.plot_heatmap(
            df=df,
            colname_feature=col_ss,
            level="pgm",
            variable_scale="logodds",
            to_path=f"{dir_plots_heatmaps_pgm}/",
            run="r_2020_02_01",
        )

## ACTUALS

In [ ]:
# Actuals heatmaps
priogrid = eval_data.fetch_priogrid()
for outcome in ["sb", "ns", "os"]:
    col_actual = f"ged_dummy_{outcome}"
    actuals = pd.DataFrame(df_pgm_b[col_actual])
    country = eval_data.fetch_country()
    actuals.reset_index(inplace=True)
    actuals = actuals.merge(priogrid, on="pg_id")
    df = actuals.merge(country, on="country_id")
    # Note that the function works with unindexed data.
    eval_plot.plot_heatmap(
        df=df,
        colname_feature=col_actual,
        level="pgm",
        run="Country proportions",
        variable_scale="logodds",
        to_path=f"{dir_plots_heatmaps_cm}/",
    )

In [ ]:
# Map of actuals (decay)
for outcome in ["sb", "ns", "os"]:
    col = f"decay_12_cw_ged_dummy_{outcome}_0"
    for t in [480]:
        df = dbutils.db_to_df(
            fqtable="launched.transforms_pgm_imp_1",
            columns=[col],
            ids=["month_id", "pg_id"],
        )
        date_str = mapdata.df_m.loc[t, "date_str"]
        fancy.plot_map(
            mapdata=mapdata,
            s_patch=df[col],
            t=t,
            run_id="Conflict history at 2019-12",
            # title=f"{pgm_column}, {date_str}",
            logodds=False,
            path=os.path.join(dir_plots_maps_pgm, f"{col}_{t}.png"),
            bbox="mainland_africa",
            textsize=18
        )


## Feature importances
Unfortunately getting feature importances forced me to rsync the full run to my local dir_runs and get the feature importances here, which makes this part not portable and so redo_featimp is False by default. 
The feature importances are in the database though so running these cells will get them to featimps_cm_c and featimps_pgm_c contain a list of feature importance dicts.

In [ ]:
def persist_featimps(featimps, fqtable):
    q_create = f"DROP TABLE IF EXISTS {fqtable}; CREATE TABLE {fqtable} (featimp JSON);"
    dbutils.execute_query(q_create)
    for featimp in featimps:
        q_insert = f"INSERT INTO {fqtable} (featimp) VALUES (:featimp)"
        params = {"featimp": json.dumps(featimp)}
        dbutils.execute_query(q_insert, **params)

def get_featimps_from_db(fqtable):
    rows = dbutils.get_query_rows(f"SELECT * FROM {fqtable};")
    return [row["featimp"] for row in rows]

In [ ]:
%%time
# Instead of the get_feature_importances_remote() call, which is portable but very slow
# rsync the full run to your local machine and run this local version instead.
# Writes the feature importances to the database 
if redo_featimp:
    dir_runs = config.CONFIG["dirs"]["dir_runs"]
    featimps_cm_c = fi.get_feature_importances_par(os.path.join(dir_runs, "cm_r_2020_02_02"))
    persist_featimps(featimps_cm_c, "newpipe.featimps_cm_c")
    featimps_pgm_c = fi.get_feature_importances_par(os.path.join(dir_runs, "pgm_r_2020_02_02"))
    persist_featimps(featimps_pgm_c, "newpipe.featimps_pgm_c")

In [ ]:
def fix_global_featimp_names(featimps):
    basemodels = ["vdem", "reign", "wdi_all", "all"]
    featimps = copy.deepcopy(featimps)
    for bm in basemodels:
        for outcome in ["sb", "ns", "os"]:
            oldname = f"{bm}_{outcome}"
            newname = f"{outcome}_{bm}_glob"
            for featimp in featimps:
                name, step = featimp["name"].split(".")
                if name == oldname:
                    featimp["name"] = f"{newname}.{step}"
    return featimps

def fix_misc_featimp_names(featimps):
    renames = {
        "sbonset24_" : "sb_onset24_",
        "nsonset24_" : "ns_onset24_",
        "osonset24_" : "os_onset24_",
        "sb100_all": "sb_100_all",
        "ns100_all": "ns_100_all",
        "os100_all": "os_100_all",
    }
    for featimp in featimps:
        for key, value in renames.items():
            featimp["name"] = featimp["name"].replace(key, value)
    return featimps

In [ ]:
# Get featimps from db
featimps_cm_c = get_featimps_from_db("newpipe.featimps_cm_c")
featimps_pgm_c = get_featimps_from_db("newpipe.featimps_pgm_c")
featimps_cm_c_global = get_featimps_from_db("newpipe.featimps_cm_c_global")

# Fix names
featimps_cm_c_global = fix_global_featimp_names(featimps_cm_c_global)
featimps_cm_c = fix_misc_featimp_names(featimps_cm_c)

featimps_cm_c = featimps_cm_c + featimps_cm_c_global

In [ ]:
def organise_featimp_by_model(featimps, namestrips):
    featimps_by_model = dict()
    for featimp in featimps:
        name, step = featimp_name_and_step(featimp, namestrips)
        step = int(step)

        if name not in featimps_by_model.keys():
            featimps_by_model[name] = dict()

        featimps_by_model[name][step] = featimp["feature_importances"]

    return featimps_by_model

def featimp_name_and_step(featimp, namestrips):
    # Get the model name
    name = featimp["name"]
    # Strip out namestrips if any passed
    if namestrips:
        for ns in namestrips:
            name = name.replace(ns, "")

    # Escape underscores in name
    name, step = name.split(".")
    return name, step

In [ ]:
namestrips = ["_rf_shift_fewsteps", "cm_africa_1_C_train.", "pgm_africa_1_C_train.",]
featimps_cm_c_by_model = organise_featimp_by_model(featimps_cm_c, namestrips)
featimps_pgm_c_by_model = organise_featimp_by_model(featimps_pgm_c, namestrips)
# Assign the featimps to the model dicts
for model in models_cm:
    if model["name"] in featimps_cm_c_by_model:
        model["featimps"] = featimps_cm_c_by_model[model["name"]]
    else:
        model["featimps"] = dict()
        print(f"No match for {model['name']}")

for model in models_pgm:
    if model["name"] in featimps_pgm_c_by_model:
        model["featimps"] = featimps_pgm_c_by_model[model["name"]]
    else:
        model["featimps"] = dict()
        print(f"No match for {model['name']}")

In [ ]:
def featimps_to_tex_str(featimps, name, limit_rows=25):
    df = pd.DataFrame(featimps)
    df = df.sort_values([1], ascending=False)
    df = df.sort_index(axis=1)
    df = df.iloc[0:limit_rows]
    tex = df.to_latex()
    name_escaped = model['name'].replace("_", "\_")
    tex = tex.replace("{} & ", f"{name_escaped} &")
    return tex


for model in models_cm:
    if model["featimps"]:
        tex_str = featimps_to_tex_str(model["featimps"], model["name"])
        path = os.path.join(dir_tables_featimp_cm, f"{model['name']}.tex")
        pyutils.write_str(tex_str, path)

for model in models_pgm:
    if model["featimps"]:
        tex_str = featimps_to_tex_str(model["featimps"], model["name"])
        path = os.path.join(dir_tables_featimp_pgm, f"{model['name']}.tex")
        pyutils.write_str(tex_str, path)
        

## Delta maps, fcast (c)

In [ ]:
run_id = "r_2020_02_01"

for ensemble in ensembles_cm:
    # Get current (2019) ensemble predictions.
    outcome = ensemble["outcome"]
    current = eval_data.fetch_ensemble_predictions(
        run_id=run_id, level="cm", outcome=outcome, eval_test=False
    )
    col_current = current.columns[0]
    for step in ensemble["steps"]:
        # Prepare params and slice data.
        col_ss = ensemble["col_ebma_ss"][step]
        month_index = df_cm_c.index.get_level_values(0).unique()[step]
        probs = df_cm_c.loc[slice(month_index, month_index), :][col_ss]
        ens_current = current.loc[slice(month_index, month_index), :]
        # Merge and get delta.
        df_cm_delta = ens_current.join(probs)
        col_a = df_cm_delta.columns[0]
        col_b = df_cm_delta.columns[1]
        df_cm_delta["delta"] = df_cm_delta[col_b] - df_cm_delta[col_a]
        # Plot.
        vmin, vmax = -0.5, 0.5
        fancy.plot_map(
            mapdata=mapdata,
            logodds=False,
            run_id=run_id,
            path=os.path.join(dir_plots_deltas_cm, f"{outcome}_{step}.png"),
            fig_scale=0.5,
            cmap="seismic",
            vmin=vmin,
            vmax=vmax,
            s_patch=df_cm_delta["delta"],
            t=month_index,
            size_gridborder=0,
            title=f"New ensemble {run_id} (s={step}) - old ensemble {run_id} (s={step})",
            bbox="mainland_africa",
            textsize=18
        )


for ensemble in ensembles_pgm:
    # Get current (2019) ensemble predictions.
    outcome = ensemble["outcome"]
    current = eval_data.fetch_ensemble_predictions(
        run_id=run_id, level="pgm", outcome=outcome, eval_test=False
    )
    col_current = current.columns[0]
    for step in ensemble["steps"]:
        # Prepare params and slice data.
        col_ss = ensemble["col_avg_ss"][step]
        month_index = df_pgm_c.index.get_level_values(0).unique()[step]
        probs = df_pgm_c.loc[slice(month_index, month_index), :][col_ss]
        ens_current = current.loc[slice(month_index, month_index), :]
        # Merge and get delta.
        df_pgm_delta = ens_current.join(probs)
        col_a = df_pgm_delta.columns[0]
        col_b = df_pgm_delta.columns[1]
        df_pgm_delta["delta"] = df_pgm_delta[col_b] - df_pgm_delta[col_a]
        # Plot.
        vmin, vmax = -0.5, 0.5
        fancy.plot_map(
            mapdata=mapdata,
            logodds=False,
            run_id=run_id,
            path=os.path.join(dir_plots_deltas_pgm, f"{outcome}_{step}.png"),
            fig_scale=0.5,
            cmap="seismic",
            vmin=vmin,
            vmax=vmax,
            s_patch=df_pgm_delta["delta"],
            t=month_index,
            size_gridborder=0,
            title=f"New ensemble {run_id} (s={step}) - old ensemble {run_id} (s={step})",
            bbox="mainland_africa",
            textsize=18
        )

## Feature maps

In [ ]:
# Plot the three time scales of spacetime feature for SB
cols_features_pgm = [
    "stdist_k1_t001_ged_dummy_sb",
    "stdist_k1_t10_ged_dummy_sb",
    "stdist_k1_t1_ged_dummy_sb",
]
df_pgm_features = dbutils.db_to_df(fqtable="flat.pgm_africa_1", ids=["month_id", "pg_id"], columns=cols_features_pgm)

### Space-time

In [ ]:
# Plot three space-time maps for the three different time scales
fancy.plot_stdist(
    df=df_pgm_features, 
    col="stdist_k1_t001_ged_dummy_sb", 
    t=468, 
    mapdata=mapdata, 
    title="Spacetime distance, sb, 2018-12, t-scale=0.01", 
    path=os.path.join(dir_plots_maps_pgm, "sb_stdist_t001_468.png")
)
fancy.plot_stdist(
    df=df_pgm_features, 
    col="stdist_k1_t1_ged_dummy_sb", 
    t=468, 
    mapdata=mapdata, 
    title="Spacetime distance, sb, 2018-12, t-scale=1.0", 
    path=os.path.join(dir_plots_maps_pgm, "sb_stdist_t1_468.png")
)
fancy.plot_stdist(
    df=df_pgm_features, 
    col="stdist_k1_t10_ged_dummy_sb", 
    t=468, 
    mapdata=mapdata, 
    title="Spacetime distance, sb, 2018-12, t-scale=10.0", 
    path=os.path.join(dir_plots_maps_pgm, "sb_stdist_t10_468.png")
)

In [ ]:
# Write all ensemble predictions to file for replication pack.
pairs = [
    (df_pgm_b, "df_pgm_b"),
    (df_pgm_c, "df_pgm_c"),
    (df_cm_b, "df_cm_b"),
    (df_cm_c, "df_cm_c"),
]

for this_df, name in pairs:
    path = os.path.join(dir_output, f"ensemble_predictions_{name}.csv")
    print(path)
    this_df[[col for col in this_df if "ensemble" in col]].to_csv(path)

In [ ]:
notebook_timer_end = time.time()
total_runtime = notebook_timer_end - notebook_timer_start
print(f"Total runtime: {total_runtime} seconds")